# Things you might think of implementing  
- replay buffer to learn from unconnected experiences
- train in batches to make everything more efficient.
- implement PPO, instantiating 2 actors. 1 will be the baseline (i.e. the previous update) in order to compute the ratio.
- define 2 different actors that will be specific for each agent. overfitting is possible.
- try to see whether you can use convolutions

# Code for executing in Colab

In [1]:
import sys
sys.path.append('/content/overcooked_ai/src')

In [2]:
!git clone https://github.com/cico-rial/overcooked_ai.git

Cloning into 'overcooked_ai'...
remote: Enumerating objects: 8079, done.
remote: Counting objects: 100% (1180/1180), done.
remote: Compressing objects: 100% (204/204), done.
remote: Total 8079 (delta 1040), reused 980 (delta 976), pack-reused 6899 (from 4)
Receiving objects: 100% (8079/8079), 567.43 MiB | 23.15 MiB/s, done.
Resolving deltas: 100% (4542/4542), done.
Updating files: 100% (448/448), done.


In [3]:
!sudo apt-get update -y
!sudo apt-get install python3.10 python3.10-dev python3.10-distutils -y
!wget https://bootstrap.pypa.io/get-pip.py
!python3.10 get-pip.py

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,721 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,994 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,246 kB]
Get:13 http://archive.ubuntu.com/u

In [ ]:
import os
os.kill(os.getpid(), 9)

In [2]:
%cd overcooked_ai
# !ls
!python3.10 -m pip install -e .

/content/overcooked_ai
Obtaining file:///content/overcooked_ai
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 139.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 128.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.9/831.9 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 167.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 148.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 148.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

# Imports

In [1]:
from overcooked_ai_py.mdp.overcooked_env import OvercookedEnv, Overcooked
from overcooked_ai_py.mdp.overcooked_mdp import OvercookedGridworld, OvercookedState
from overcooked_ai_py.visualization.state_visualizer import StateVisualizer
from overcooked_ai_py.mdp.actions import Action
from overcooked_ai_py.agents.agent import Agent, AgentPair, RandomAgent
from overcooked_ai_py.agents.benchmarking import AgentEvaluator
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist, fashion_mnist
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, BatchNormalization, Activation, Concatenate
from tensorflow.keras.models import Model
from tqdm.notebook import tqdm
from typing import Tuple, List, Dict
import os

In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

TensorFlow version: 2.19.0
Num GPUs Available: 0


# Useful classes

Policy class: NN for computing the probability distribution of the actions to take.

In [3]:
class Policy(Model):
    def __init__(self, input_shape, num_actions, optimizer, epsilon = 0.05):
        super().__init__()
        self.input_shape = input_shape
        self.num_actions = num_actions
        self.optimizer = optimizer
        self.epsilon = epsilon
        self.input_a = Input(shape=(self.input_shape))
        self.input_b = Input(shape=(self.input_shape))
        # self.concatenate_inputs = Concatenate()
        self.dense_1 = layers.Dense(128, activation='tanh')
        self.dense_2 = layers.Dense(256, activation='tanh')
        self.dense_3 = layers.Dense(256, activation='tanh')
        self.dense_4 = layers.Dense(128, activation='tanh')
        self.policy_a = layers.Dense(self.num_actions, activation='softmax', name="policy_a")
        self.policy_b = layers.Dense(self.num_actions, activation='softmax', name="policy_b")
        self.build_model()

    # def preprocess(self, obs):
    #     agent_1_obs = obs['both_agent_obs'][0]
    #     agent_2_obs = obs['both_agent_obs'][1]
    #     return (tf.expand_dims(agent_1_obs, axis=0), tf.expand_dims(agent_2_obs, axis=0))
    
    def preprocess(self, obs):
        if isinstance(obs, Tuple):
            obs = [obs] # to handle the case where obs_batch is a single observation

        obs_1, obs_2 = zip(*obs)
        obs_batch = tf.concat([tf.stack(obs_1), tf.stack(obs_2)], axis=-1)
        return obs_batch


    def call(self, obs, training=False):
        x = self.preprocess(obs)
        # x = self.concatenate_inputs(x)
        x = self.dense_1(x)
        x = self.dense_2(x)
        x = self.dense_3(x)
        x = self.dense_4(x)
        policy_a = self.policy_a(x)
        policy_b = self.policy_b(x)
        return (policy_a, policy_b)

    def build_model(self):
        # computing a forward pass in order to automatically build the model
        dummy_input = (
            tf.zeros((1, 96)),
            tf.zeros((1, 96))
        )
        _ = self(dummy_input)

    def train_step(self, delta, obs: Tuple, action: Tuple[int,int]):
        # update t with t + alpha_t*delta*grad_pi^(A|S) where A is the action taken before reaching St+1
        with tf.GradientTape() as tape:
            pi = self.call(obs, training=True)
            log_pi = tf.math.log(pi)
            pi_a = log_pi[0][..., action[0]] + log_pi[1][..., action[1]] # π(A|S), computing the sum of the probability of the best actions

        grad_pi_a = tape.gradient(pi_a, self.trainable_weights)
        processed_gradient = [-tf.squeeze(delta)*grad for grad in grad_pi_a]
        self.optimizer.apply_gradients(zip(processed_gradient, self.trainable_weights))
    
    def train_batch(self, deltas_batch: tf.Tensor, obs_batch, actions_batch):
        # update t with t + alpha_t*delta*grad_pi^(A|S) where A is the action taken before reaching St+1
        with tf.GradientTape() as tape:
            pi = self.call(obs_batch, training=True)
            log_pi = tf.math.log(pi)
            pi_a1 = tf.gather(log_pi[0], actions_batch[:, 0], axis=1, batch_dims=1)
            pi_a2 = tf.gather(log_pi[1], actions_batch[:, 1], axis=1, batch_dims=1)
            # Now compute the weighted sum over the batch
            pi_a = -tf.reduce_sum(tf.squeeze(deltas_batch) * (pi_a1 + pi_a2))

        grad_pi_a = tape.gradient(pi_a, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grad_pi_a, self.trainable_weights))

    def train_batch_PPO(self, deltas_batch: tf.Tensor, obs_batch, actions_batch, old_policy):
        with tf.GradientTape() as tape:
            pi = self.call(obs_batch, training=True)
            old_pi = old_policy.call(obs_batch)
            pi_ratio_1 = pi[0] / old_pi[0] + 1e-8 # to avoid numerical instability
            pi_ratio_2 = pi[1] / old_pi[1] + 1e-8 # to avoid numerical instability
            pi_clipped_ratio_1 = tf.clip_by_value(pi_ratio_1, 1 - self.epsilon, 1 + self.epsilon)
            pi_clipped_ratio_2 = tf.clip_by_value(pi_ratio_2, 1 - self.epsilon, 1 + self.epsilon)
            pi_ratio_advantage_1 = pi_ratio_1*deltas_batch
            pi_ratio_advantage_2 = pi_ratio_2*deltas_batch
            pi_clipped_ratio_advantage_1 = pi_clipped_ratio_1*deltas_batch
            pi_clipped_ratio_advantage_2 = pi_clipped_ratio_2*deltas_batch
            L = 0
            for i in range(len(actions_batch)):
                L += min(pi_ratio_advantage_1[i][actions_batch[i][0]], pi_clipped_ratio_advantage_1[i][actions_batch[i][0]])
                L += min(pi_ratio_advantage_2[i][actions_batch[i][1]], pi_clipped_ratio_advantage_2[i][actions_batch[i][1]])
            loss = -L

        grad_loss = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grad_loss, self.trainable_weights))

ValueFunctionApproximator class: NN for approximating the value function of a given state.

In [4]:
class ValueFunctionApproximator(Model):
    def __init__(self, input_shape, optimizer):
        super().__init__()
        self.input_shape = input_shape
        self.optimizer = optimizer
        self.input_a = Input(shape=(self.input_shape))
        self.input_b = Input(shape=(self.input_shape))
        # self.concatenate_inputs = Concatenate()
        self.dense_1 = layers.Dense(128, activation='tanh')
        self.dense_2 = layers.Dense(256, activation='tanh')
        self.dense_3 = layers.Dense(256, activation='tanh')
        self.dense_4 = layers.Dense(128, activation='tanh')
        # self.value_function = layers.Dense(1, activation='relu', name="value_function")
        self.value_function = layers.Dense(1, name="value_function")
        self.build_model()

    
    def preprocess(self, obs):
        if isinstance(obs, Tuple):
            obs = [obs] # to handle the case where obs_batch is a single observation

        obs_1, obs_2 = zip(*obs)
        obs_batch = tf.concat([tf.stack(obs_1), tf.stack(obs_2)], axis=-1)
        return obs_batch


    def call(self, obs: Tuple, training=False):
        x = self.preprocess(obs)
        # x = self.concatenate_inputs(x)
        x = self.dense_1(x)
        x = self.dense_2(x)
        x = self.dense_3(x)
        x = self.dense_4(x)
        value_function = self.value_function(x)
        return value_function

    def build_model(self):
        # computing a forward pass in order to automatically build the model
        dummy_input = (
            tf.zeros((1, 96)),
            tf.zeros((1, 96))
        )
        _ = self(dummy_input)

    def train_step(self, delta, obs: Tuple):
        # update w with w + alpha_w*delta*grad_v^(St)
        with tf.GradientTape() as tape:
            state_value = self.call(obs, training=True)

        grad_state_value = tape.gradient(state_value, self.trainable_weights)
        processed_gradient = [-tf.squeeze(delta)*grad for grad in grad_state_value]
        self.optimizer.apply_gradients(zip(processed_gradient, self.trainable_weights))

    def train_batch(self, deltas_batch: tf.Tensor, obs_batch): # deltas is a tf.Tensor of shape (batch_size,1)
        # update w with w + alpha_w*grad_v^(St)*delta
        with tf.GradientTape() as tape:
            state_value = self.call(obs_batch, training=True)
            processed_state_value = -deltas_batch * state_value

        grad_state_value = tape.gradient(processed_state_value, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grad_state_value, self.trainable_weights))

Agent class: inherits from Agent class found in agent.py

In [5]:
class MyAgent(Agent):
    """
    This class is more a couple of actors since we use shared networks and the output are 2!!!
    For now let's treat it like a single player identified by self.index
    """
    def __init__(self, actor, old_policy, critic, idx, base_env: OvercookedEnv):
        super().__init__()
        self.actor = actor
        self.old_policy = old_policy
        self.critic = critic
        self.idx = idx
        if not self.idx in [0,1]:
            raise AssertionError("The index of the agent must be either 0 or 1!")
        self.base_env = base_env

    def action(self, obs):
        """
        obs: preprocessed observation (or overcookedstate)
        We want to output the action given the state. can use a NN!
        should return a tuple (Action, Dict)
        Dict should contain info about the action ('action_probs': numpy array)
        """
        if isinstance(obs, OvercookedState):
            # this is useful for translating the OvercookedState
            # into observation that can be fed into the NN.
            state = obs
            obs_from_state = self.base_env.featurize_state_mdp(state)
            # obs = {'both_agent_obs': obs_from_state, 'overcooked_state': state}
            obs = (obs_from_state[0],obs_from_state[1])

        action_probs = self.actor.call(obs)[self.idx].numpy()
        # if np.random.random() > self.epsilon:
        #     action = Action.argmax(action_probs) # greedy selection
        # else:
        #     action_idx = np.random.choice(range(0,6), size=1)[0]
        #     action = Action.INDEX_TO_ACTION[action_idx] # random exploration
        sample = tf.random.categorical(tf.math.log(action_probs), num_samples = 1)
        action_idx = tf.squeeze(sample, axis=-1)[0].numpy()
        action = Action.INDEX_TO_ACTION[action_idx]
        return (action, {'action_probs': action_probs})

    def actions(self, obss):
        """
        Look at the documentation of the Agent class
        """
        pass

    def update(self, obs, reward):
        """
        What do we need to update?
        """
        pass

    def update_old_policy(self):
        self.old_policy.set_weights(self.actor.get_weights())

    # def decay_epsilon(self, decay):
    #     if self.epsilon - decay <= 0.05:
    #         self.epsilon = 0.05
    #     else:
    #         self.epsilon -= decay

# Let's start coding!

Let's define our environment:

In [6]:
number_of_frames = 400
layout_name = "cramped_room"
base_mdp = OvercookedGridworld.from_layout_name(layout_name=layout_name) #or other layout
base_env = OvercookedEnv.from_mdp(base_mdp, info_level=0, horizon=number_of_frames)
env = Overcooked(base_env=base_env, featurize_fn=base_env.featurize_state_mdp)

Let's instantiate our networks and the hyperparameters:

In [7]:
alpha_w = 1e-5
alpha_t = 1e-6
critic_optimizer = Adam(learning_rate=alpha_w)
actor_optimizer = Adam(learning_rate=alpha_t)

In [8]:
input_shape = env.observation_space._shape

shared_actor = Policy(
    input_shape=input_shape,
    num_actions=Action.NUM_ACTIONS,
    optimizer=actor_optimizer
    )

shared_critic = ValueFunctionApproximator(
    input_shape=input_shape,
    optimizer=critic_optimizer
    )

In [10]:
path_critic = "networks/critic/"
path_actor = "networks/actor/"

In [10]:
os.makedirs(path_critic, exist_ok=True)
os.makedirs(path_actor, exist_ok=True)
print(f"Created directories: {path_critic} and {path_actor}")

Created directories: networks/critic/ and networks/actor/


In [11]:
# shared_critic.load_weights(path_critic + "shared_critic.weights.h5")
# shared_actor.load_weights(path_actor + "shared_actor.weights.h5")

Let's instantiate our agents:

In [12]:
agent_1 = MyAgent(
    actor=shared_actor,
    critic=shared_critic,
    idx=0,
    base_env=base_env,
)
agent_2 = MyAgent(
    actor=shared_actor,
    critic=shared_critic,
    idx=1,
    base_env=base_env,
)

Let's define our training loop! Let's go for now for a bootstrapping method.

In [13]:
# shared_critic.load_weights(path_critic + "shared_critic_deep_batch.weights.h5")
# shared_actor.load_weights(path_actor + "shared_actor_deep_batch.weights.h5")

# shared_critic.save_weights(path_critic + "shared_critic_deep_batch.weights.h5")
# shared_actor.save_weights(path_actor + "shared_actor_deep_batch.weights.h5")

In [14]:
number_of_episodes = 1000
number_of_epochs = 4
batch_size = 10
last_actor_call = 0 # debugging
stop_calling = False # debugging
average_reward = 0
gamma = 0.95
buffer_replay = {'observations': [],
                 'actions': [],
                 'deltas': []}
try:
    for episode in range(1, number_of_episodes + 1):
        t = 0
        obs = env.reset()
        obs = obs['both_agent_obs'] # BRO I WAS TOO LAZY TO OVERRIDE OBS. DON'T FORGET IT

        done = False
        shaped_reward_factor = 1 # multiplicative factor for discounting shaped reward.
        discount_rate = 1 # will decrease the shaped reward every time
        cumulative_reward = 0
        updated = False

        if stop_calling: # debugging
            break # debugging

        if (episode) % 50 == 0:
            shared_critic.save_weights(path_critic + "shared_critic_exp_1.weights.h5")
            shared_actor.save_weights(path_actor + "shared_actor_exp_1.weights.h5")

        while not done:
            action1 = agent_1.action(obs)
            action2 = agent_2.action(obs)
            player_1_action = Action.ACTION_TO_INDEX[action1[0]]
            player_2_action = Action.ACTION_TO_INDEX[action2[0]]
            action = (player_1_action, player_2_action)

            new_obs, reward, done, env_info = env.step(action)
            shaped_reward = sum(env_info['shaped_r_by_agent']) # let's use shaped reward for learning how to play first.

            new_obs = new_obs['both_agent_obs']

            if shaped_reward != 0:
                shaped_reward = shaped_reward * shaped_reward_factor # discounting the shaped reward
                shaped_reward_factor *= discount_rate

            total_reward = reward + shaped_reward

            cumulative_reward += total_reward

            # compute delta = R + v^(St+1) - v^(St) where v^(St+1) = 0 if done
            if done:
                delta = total_reward - shared_critic.call(obs)
            else:
                delta = total_reward + gamma*shared_critic.call(new_obs) - shared_critic.call(obs)

            if not stop_calling: # debugging
                last_actor_call = shared_actor.call(obs) # debugging

            # adding experience to the buffer replay if it is a good experience
            # let's try to understand what is a good experience (high reward, high delta...)
            if total_reward > 0:
                buffer_replay['observations'].append(obs)
                buffer_replay['deltas'].append(delta)
                buffer_replay['actions'].append(action)
            
            if len(buffer_replay['observations']) > 200:
                print(f"Performing {number_of_epochs} epochs of stocastic gradient descent on the replay buffer.")
                for epoch in range(1, number_of_epochs + 1):
                    num_batches = len(buffer_replay['observations']) // batch_size
                    shuffled_indices = tf.random.shuffle(tf.range(len(buffer_replay['observations'])))
                    for batch in range(num_batches):
                        if batch == num_batches: # last batch
                            idx = shuffled_indices[batch*batch_size:]
                        else:
                            idx = shuffled_indices[batch*batch_size:(batch+1)*batch_size]

                        deltas_batch = tf.squeeze(tf.gather(buffer_replay['deltas'], idx), axis=-1)
                        actions_batch = tf.gather(buffer_replay['actions'], idx)
                        observations_batch = tf.gather(buffer_replay['observations'], idx)

                        # shared_critic.train_batch(deltas_batch, observations_batch)
                        shared_actor.train_batch(deltas_batch, observations_batch, actions_batch)
                    print(f"Epoch {epoch} terminated.")
                buffer_replay['observations'] = buffer_replay['observations'][50:]
                buffer_replay['actions'] = buffer_replay['actions'][50:]
                buffer_replay['deltas'] = buffer_replay['deltas'][50:]

            if delta != 0:
                updated = True
                # update w with w + alpha_w*delta*grad_v^(St)
                shared_critic.train_step(delta, obs)
                # update t with t + alpha_t*delta*grad_pi^(A|S) where A is the action taken before reaching St+1
                shared_actor.train_step(delta, obs, action)

                # DEBUGGING
                if not np.sum(shared_actor.call(obs)[1].numpy()) > 0.9:
                    print(delta)
                    stop_calling = True
                    break
                # END DEBUGGING
            # update state (obs = new_obs)
            obs = new_obs

            # think about training the critic by itself for a while
            t += 1

        average_reward = 1/(episode)*( cumulative_reward + (episode-1)*average_reward)
        print(f"Episode [{episode:>3d}] terminated at timestep {t}. cumulative reward: {cumulative_reward:>3d}. updated: {updated}. avg reward: {round(average_reward, 3)}")
        # shared_critic.save_weights(path_critic + "shared_critic.weights.h5")
        # shared_actor.save_weights(path_actor + "shared_actor.weights.h5")
except KeyboardInterrupt:
    shared_critic.save_weights(path_critic + "shared_critic_exp_1.weights.h5")
    shared_actor.save_weights(path_actor + "shared_actor_exp_1.weights.h5")
    print("User interrupted training. Saving weights")

Episode [  1] terminated at timestep 400. cumulative reward:   6. updated: True. avg reward: 6.0
Episode [  2] terminated at timestep 400. cumulative reward:   3. updated: True. avg reward: 4.5
Episode [  3] terminated at timestep 400. cumulative reward:   6. updated: True. avg reward: 5.0
Episode [  4] terminated at timestep 400. cumulative reward:   3. updated: True. avg reward: 4.5
Episode [  5] terminated at timestep 400. cumulative reward:   6. updated: True. avg reward: 4.8
Episode [  6] terminated at timestep 400. cumulative reward:   6. updated: True. avg reward: 5.0
Episode [  7] terminated at timestep 400. cumulative reward:   8. updated: True. avg reward: 5.429
Episode [  8] terminated at timestep 400. cumulative reward:   3. updated: True. avg reward: 5.125
Episode [  9] terminated at timestep 400. cumulative reward:   3. updated: True. avg reward: 4.889
Episode [ 10] terminated at timestep 400. cumulative reward:   0. updated: True. avg reward: 4.4
Episode [ 11] terminated

In [ ]:
# Here we create an evaluator for the cramped_room layout
layout = "cramped_room"
ae = AgentEvaluator.from_layout_name(mdp_params={"layout_name": layout, "old_dynamics": True},
                                     env_params={"horizon": 400})

ap = AgentPair(agent_1, agent_2)

trajs = ae.evaluate_agent_pair(ap, 10)
# trajs = ae.evaluate_human_model_pair(1)

StateVisualizer().display_rendered_trajectory(trajs, ipython_display=True)

Avg rew: 4.00 (std: 8.00, se: 2.53); avg len: 400.00; : 100%|██████████| 10/10 [00:58<00:00,  5.86s/it]


interactive(children=(IntSlider(value=0, description='timestep', max=399), Output()), _dom_classes=('widget-in…

# Without experience replay

In [20]:
number_of_frames = 400
layout_name = "cramped_room"
base_mdp = OvercookedGridworld.from_layout_name(layout_name=layout_name) #or other layout
base_env = OvercookedEnv.from_mdp(base_mdp, info_level=0, horizon=number_of_frames)
env = Overcooked(base_env=base_env, featurize_fn=base_env.featurize_state_mdp)

In [21]:
alpha_w = 1e-5
alpha_t = 1e-6
critic_optimizer = Adam(learning_rate=alpha_w)
actor_optimizer = Adam(learning_rate=alpha_t)

In [22]:
input_shape = env.observation_space._shape

shared_actor = Policy(
    input_shape=input_shape,
    num_actions=Action.NUM_ACTIONS,
    optimizer=actor_optimizer
    )

shared_critic = ValueFunctionApproximator(
    input_shape=input_shape,
    optimizer=critic_optimizer
    )

In [23]:
agent_1 = MyAgent(
    actor=shared_actor,
    critic=shared_critic,
    idx=0,
    base_env=base_env,
)
agent_2 = MyAgent(
    actor=shared_actor,
    critic=shared_critic,
    idx=1,
    base_env=base_env,
)

In [24]:
number_of_episodes = 300
number_of_epochs = 4
batch_size = 10
last_actor_call = 0 # debugging
stop_calling = False # debugging
average_reward = 0
gamma = 0.95

try:
    for episode in range(1, number_of_episodes + 1):
        t = 0
        obs = env.reset()
        obs = obs['both_agent_obs'] # BRO I WAS TOO LAZY TO OVERRIDE OBS. DON'T FORGET IT

        done = False
        shaped_reward_factor = 1 # multiplicative factor for discounting shaped reward.
        discount_rate = 1 # will decrease the shaped reward every time
        cumulative_reward = 0
        updated = False

        if stop_calling: # debugging
            break # debugging

        if (episode) % 50 == 0:
            shared_critic.save_weights(path_critic + "shared_critic_exp_1_2.weights.h5")
            shared_actor.save_weights(path_actor + "shared_actor_exp_1_2.weights.h5")

        while not done:
            action1 = agent_1.action(obs)
            action2 = agent_2.action(obs)
            player_1_action = Action.ACTION_TO_INDEX[action1[0]]
            player_2_action = Action.ACTION_TO_INDEX[action2[0]]
            action = (player_1_action, player_2_action)

            new_obs, reward, done, env_info = env.step(action)
            shaped_reward = sum(env_info['shaped_r_by_agent']) # let's use shaped reward for learning how to play first.

            new_obs = new_obs['both_agent_obs']

            if shaped_reward != 0:
                shaped_reward = shaped_reward * shaped_reward_factor # discounting the shaped reward
                shaped_reward_factor *= discount_rate

            total_reward = reward + shaped_reward

            cumulative_reward += total_reward

            # compute delta = R + v^(St+1) - v^(St) where v^(St+1) = 0 if done
            if done:
                delta = total_reward - shared_critic.call(obs)
            else:
                delta = total_reward + gamma*shared_critic.call(new_obs) - shared_critic.call(obs)

            if not stop_calling: # debugging
                last_actor_call = shared_actor.call(obs) # debugging

            # adding experience to the buffer replay if it is a good experience
            

            if delta != 0:
                updated = True
                # update w with w + alpha_w*delta*grad_v^(St)
                shared_critic.train_step(delta, obs)
                # update t with t + alpha_t*delta*grad_pi^(A|S) where A is the action taken before reaching St+1
                shared_actor.train_step(delta, obs, action)

                # DEBUGGING
                if not np.sum(shared_actor.call(obs)[1].numpy()) > 0.9:
                    print(delta)
                    stop_calling = True
                    break
                # END DEBUGGING
            # update state (obs = new_obs)
            obs = new_obs

            # think about training the critic by itself for a while
            t += 1

        average_reward = 1/(episode)*( cumulative_reward + (episode-1)*average_reward)
        print(f"Episode [{episode:>3d}] terminated at timestep {t}. cumulative reward: {cumulative_reward:>3d}. updated: {updated}. avg reward: {round(average_reward, 3)}")
        # shared_critic.save_weights(path_critic + "shared_critic.weights.h5")
        # shared_actor.save_weights(path_actor + "shared_actor.weights.h5")
except KeyboardInterrupt:
    shared_critic.save_weights(path_critic + "shared_critic_exp_1_2.weights.h5")
    shared_actor.save_weights(path_actor + "shared_actor_exp_1_2.weights.h5")
    print("User interrupted training. Saving weights")

Episode [  1] terminated at timestep 400. cumulative reward:   0. updated: True. avg reward: 0.0
Episode [  2] terminated at timestep 400. cumulative reward:   9. updated: True. avg reward: 4.5
Episode [  3] terminated at timestep 400. cumulative reward:   3. updated: True. avg reward: 4.0
Episode [  4] terminated at timestep 400. cumulative reward:   3. updated: True. avg reward: 3.75
Episode [  5] terminated at timestep 400. cumulative reward:   6. updated: True. avg reward: 4.2
Episode [  6] terminated at timestep 400. cumulative reward:   6. updated: True. avg reward: 4.5
Episode [  7] terminated at timestep 400. cumulative reward:  11. updated: True. avg reward: 5.429
Episode [  8] terminated at timestep 400. cumulative reward:  14. updated: True. avg reward: 6.5
Episode [  9] terminated at timestep 400. cumulative reward:   6. updated: True. avg reward: 6.444
Episode [ 10] terminated at timestep 400. cumulative reward:   6. updated: True. avg reward: 6.4
Episode [ 11] terminated 

# Let's see the RandomAgent average reward to see whether my algorithm is learning something or not.

In [ ]:
random_agent_1 = RandomAgent(all_actions=True)
random_agent_2 = RandomAgent(all_actions=True)
number_of_episodes = 100
average_reward = 0
for episode in range(1, number_of_episodes + 1):
    t = 0
    obs = env.reset()
    done = False
    cumulative_reward = 0

    while not done:
        action1 = random_agent_1.action(obs)
        action2 = random_agent_2.action(obs)
        player_1_action = Action.ACTION_TO_INDEX[action1[0]]
        player_2_action = Action.ACTION_TO_INDEX[action2[0]]
        action = (player_1_action, player_2_action)

        new_obs, reward, done, env_info = env.step(action)
        shaped_reward = sum(env_info['shaped_r_by_agent']) # let's use shaped reward for learning how to play first.

        total_reward = reward + shaped_reward

        cumulative_reward += total_reward

        obs = new_obs

        # think about training the critic by itself for a while
        t += 1

    average_reward = 1/(episode)*( cumulative_reward + (episode-1)*average_reward)
    print(f"Episode [{episode:>3d}] terminated at timestep {t}. cumulative reward: {cumulative_reward:>3d}. avg reward: {round(average_reward, 3)}")

Random baseline: avg reward = 8

# Trial ordered

In [25]:
number_of_frames = 400
layout_name = "cramped_room"
base_mdp = OvercookedGridworld.from_layout_name(layout_name=layout_name) #or other layout
base_env = OvercookedEnv.from_mdp(base_mdp, info_level=0, horizon=number_of_frames)
env = Overcooked(base_env=base_env, featurize_fn=base_env.featurize_state_mdp)

In [26]:
alpha_w = 1e-5
alpha_t = 1e-6
critic_optimizer = Adam(learning_rate=alpha_w)
actor_optimizer = Adam(learning_rate=alpha_t)

In [27]:
input_shape = env.observation_space._shape

shared_actor = Policy(
    input_shape=input_shape,
    num_actions=Action.NUM_ACTIONS,
    optimizer=actor_optimizer
    )

shared_critic = ValueFunctionApproximator(
    input_shape=input_shape,
    optimizer=critic_optimizer
    )

In [28]:
path_critic = "networks/critic/"
path_actor = "networks/actor/"

In [29]:
os.makedirs(path_critic, exist_ok=True)
os.makedirs(path_actor, exist_ok=True)
print(f"Created directories: {path_critic} and {path_actor}")

Created directories: networks/critic/ and networks/actor/


In [30]:
agent_1 = MyAgent(
    actor=shared_actor,
    critic=shared_critic,
    idx=0,
    base_env=base_env,
)
agent_2 = MyAgent(
    actor=shared_actor,
    critic=shared_critic,
    idx=1,
    base_env=base_env,
)

In [31]:
number_of_episodes = 300
number_of_epochs = 4
batch_size = 10
last_actor_call = 0 # debugging
stop_calling = False # debugging
average_reward = 0
gamma = 0.95
buffer_replay = {'observations': [],
                 'actions': [],
                 'deltas': []}
try:
    for episode in range(1, number_of_episodes + 1):
        t = 0
        obs = env.reset()

        done = False
        shaped_reward_factor = 1 # multiplicative factor for discounting shaped reward.
        discount_rate = 1 # will decrease the shaped reward every time
        cumulative_reward = 0
        updated = False

        if stop_calling: # debugging
            break # debugging

        if (episode) % 50 == 0:
            shared_critic.save_weights(path_critic + "shared_critic_exp_2.weights.h5")
            shared_actor.save_weights(path_actor + "shared_actor_exp_2.weights.h5")

        while not done:
            action1 = agent_1.action(obs['both_agent_obs'])
            action2 = agent_2.action(obs['both_agent_obs'])
            player_1_action = Action.ACTION_TO_INDEX[action1[0]]
            player_2_action = Action.ACTION_TO_INDEX[action2[0]]
            action = (player_1_action, player_2_action)

            if obs['other_agent_env_idx'] == 0:
                my_obs = (obs['both_agent_obs'][1],obs['both_agent_obs'][0])
                obs['both_agent_obs'] = my_obs # BRO I WAS TOO LAZY TO OVERRIDE OBS. DON'T FORGET IT
                action = (player_2_action, player_1_action)
            else:
                action = (player_1_action, player_2_action)

            new_obs, reward, done, env_info = env.step(action)
            shaped_reward = sum(env_info['shaped_r_by_agent']) # let's use shaped reward for learning how to play first.

            if shaped_reward != 0:
                shaped_reward = shaped_reward * shaped_reward_factor # discounting the shaped reward
                shaped_reward_factor *= discount_rate

            total_reward = reward + shaped_reward

            cumulative_reward += total_reward

            # compute delta = R + v^(St+1) - v^(St) where v^(St+1) = 0 if done
            if done:
                delta = total_reward - shared_critic.call(obs['both_agent_obs'])
            else:
                delta = total_reward + gamma*shared_critic.call(new_obs['both_agent_obs']) - shared_critic.call(obs['both_agent_obs'])

            if not stop_calling: # debugging
                last_actor_call = shared_actor.call(obs['both_agent_obs']) # debugging

            # adding experience to the buffer replay if it is a good experience
            # let's try to understand what is a good experience (high reward, high delta...)
            if total_reward > 0:
                buffer_replay['observations'].append(obs['both_agent_obs'])
                buffer_replay['deltas'].append(delta)
                buffer_replay['actions'].append(action)
            
            if len(buffer_replay['observations']) > 200:
                print(f"Performing {number_of_epochs} epochs of stocastic gradient descent on the replay buffer.")
                for epoch in range(1, number_of_epochs + 1):
                    num_batches = len(buffer_replay['observations']) // batch_size
                    shuffled_indices = tf.random.shuffle(tf.range(len(buffer_replay['observations'])))
                    for batch in range(num_batches):
                        if batch == num_batches: # last batch
                            idx = shuffled_indices[batch*batch_size:]
                        else:
                            idx = shuffled_indices[batch*batch_size:(batch+1)*batch_size]

                        deltas_batch = tf.squeeze(tf.gather(buffer_replay['deltas'], idx), axis=-1)
                        actions_batch = tf.gather(buffer_replay['actions'], idx)
                        observations_batch = tf.gather(buffer_replay['observations'], idx)

                        # shared_critic.train_batch(deltas_batch, observations_batch)
                        shared_actor.train_batch(deltas_batch, observations_batch, actions_batch)
                    print(f"Epoch {epoch} terminated.")
                buffer_replay['observations'] = buffer_replay['observations'][50:]
                buffer_replay['actions'] = buffer_replay['actions'][50:]
                buffer_replay['deltas'] = buffer_replay['deltas'][50:]

            if delta != 0:
                updated = True
                # update w with w + alpha_w*delta*grad_v^(St)
                shared_critic.train_step(delta, obs['both_agent_obs'])
                # update t with t + alpha_t*delta*grad_pi^(A|S) where A is the action taken before reaching St+1
                shared_actor.train_step(delta, obs['both_agent_obs'], action)

                # DEBUGGING
                if not np.sum(shared_actor.call(obs['both_agent_obs'])[1].numpy()) > 0.9:
                    print(delta)
                    stop_calling = True
                    break
                # END DEBUGGING
            # update state (obs = new_obs)
            obs = new_obs

            # think about training the critic by itself for a while
            t += 1

        average_reward = 1/(episode)*( cumulative_reward + (episode-1)*average_reward)
        print(f"Episode [{episode:>3d}] terminated at timestep {t}. cumulative reward: {cumulative_reward:>3d}. updated: {updated}. avg reward: {round(average_reward, 3)}")
        # shared_critic.save_weights(path_critic + "shared_critic.weights.h5")
        # shared_actor.save_weights(path_actor + "shared_actor.weights.h5")
except KeyboardInterrupt:
    shared_critic.save_weights(path_critic + "shared_critic_exp_2.weights.h5")
    shared_actor.save_weights(path_actor + "shared_actor_exp_2.weights.h5")
    print("User interrupted training. Saving weights")

Episode [  1] terminated at timestep 400. cumulative reward:   6. updated: True. avg reward: 6.0
Episode [  2] terminated at timestep 400. cumulative reward:  14. updated: True. avg reward: 10.0
Episode [  3] terminated at timestep 400. cumulative reward:   3. updated: True. avg reward: 7.667
Episode [  4] terminated at timestep 400. cumulative reward:   3. updated: True. avg reward: 6.5
Episode [  5] terminated at timestep 400. cumulative reward:   8. updated: True. avg reward: 6.8
Episode [  6] terminated at timestep 400. cumulative reward:   3. updated: True. avg reward: 6.167
Episode [  7] terminated at timestep 400. cumulative reward:   3. updated: True. avg reward: 5.714
Episode [  8] terminated at timestep 400. cumulative reward:   6. updated: True. avg reward: 5.75
Episode [  9] terminated at timestep 400. cumulative reward:   6. updated: True. avg reward: 5.778
Episode [ 10] terminated at timestep 400. cumulative reward:  20. updated: True. avg reward: 7.2
Episode [ 11] termin

In [ ]:
# Here we create an evaluator for the cramped_room layout
layout = "cramped_room"
ae = AgentEvaluator.from_layout_name(mdp_params={"layout_name": layout, "old_dynamics": True},
                                     env_params={"horizon": 400})

ap = AgentPair(agent_1, agent_2)

trajs = ae.evaluate_agent_pair(ap, 10)
# trajs = ae.evaluate_human_model_pair(1)

StateVisualizer().display_rendered_trajectory(trajs, ipython_display=True)

# Trial with batches

In [32]:
number_of_frames = 400
layout_name = "cramped_room"
base_mdp = OvercookedGridworld.from_layout_name(layout_name=layout_name) #or other layout
base_env = OvercookedEnv.from_mdp(base_mdp, info_level=0, horizon=number_of_frames)
env = Overcooked(base_env=base_env, featurize_fn=base_env.featurize_state_mdp)

In [33]:
alpha_w = 1e-5
alpha_t = 1e-6
critic_optimizer = Adam(learning_rate=alpha_w)
actor_optimizer = Adam(learning_rate=alpha_t)

In [34]:
input_shape = env.observation_space._shape

shared_actor = Policy(
    input_shape=input_shape,
    num_actions=Action.NUM_ACTIONS,
    optimizer=actor_optimizer
    )

shared_critic = ValueFunctionApproximator(
    input_shape=input_shape,
    optimizer=critic_optimizer
    )

In [35]:
agent_1 = MyAgent(
    actor=shared_actor,
    critic=shared_critic,
    idx=0,
    base_env=base_env,
)
agent_2 = MyAgent(
    actor=shared_actor,
    critic=shared_critic,
    idx=1,
    base_env=base_env,
)

In [36]:
number_of_episodes = 500
number_of_epochs = 4
batch_size = 20
average_reward = 0
gamma = 0.95

try:
    for episode in range(1, number_of_episodes + 1):
        actions = []
        deltas = []
        observations = []

        t = 0
        obs = env.reset()
        obs = obs['both_agent_obs'] 
        
        done = False
        cumulative_reward = 0

        if (episode) % 50 == 0:
            shared_critic.save_weights(path_critic + "shared_critic_exp_3.weights.h5")
            shared_actor.save_weights(path_actor + "shared_actor_exp_3.weights.h5")

        while not done:
            action1 = agent_1.action(obs)
            action2 = agent_2.action(obs)
            player_1_action = Action.ACTION_TO_INDEX[action1[0]]
            player_2_action = Action.ACTION_TO_INDEX[action2[0]]
            action = (player_1_action, player_2_action)

            actions.append(action)
            observations.append(obs)

            new_obs, reward, done, env_info = env.step(action)
            shaped_reward = sum(env_info['shaped_r_by_agent']) # let's use shaped reward for learning how to play first.

            new_obs = new_obs['both_agent_obs']

            if shaped_reward != 0:
                shaped_reward = shaped_reward * shaped_reward_factor # discounting the shaped reward
                shaped_reward_factor *= discount_rate

            total_reward = reward + shaped_reward

            cumulative_reward += total_reward

            # compute delta = R + v^(St+1) - v^(St) where v^(St+1) = 0 if done
            if done:
                delta = total_reward - shared_critic.call(obs)
            else:
                delta = total_reward + gamma*shared_critic.call(new_obs) - shared_critic.call(obs)

            deltas.append(delta)
            
            # update state (obs = new_obs)
            obs = new_obs

            # think about training the critic by itself for a while
            t += 1

        average_reward = 1/(episode)*( cumulative_reward + (episode-1)*average_reward)
        print(f"Episode [{episode:>3d}] terminated at timestep {t}. cumulative reward: {cumulative_reward:>3d}. avg reward: {round(average_reward, 3)}")
        print(f"Performing stocastic gradient descent with {number_of_epochs} epochs.")
        for epoch in range(1, number_of_epochs + 1):
            num_batches = len(actions) // batch_size
            shuffled_indices = tf.random.shuffle(tf.range(len(actions)))
            for batch in range(num_batches):
                if batch == num_batches: # last batch
                    idx = shuffled_indices[batch*batch_size:]
                else:
                    idx = shuffled_indices[batch*batch_size:(batch+1)*batch_size]

                deltas_batch = tf.squeeze(tf.gather(deltas, idx), axis=-1)
                actions_batch = tf.gather(actions, idx)
                observations_batch = tf.gather(observations, idx)

                shared_critic.train_batch(deltas_batch, observations_batch)
                shared_actor.train_batch(deltas_batch, observations_batch, actions_batch)
            print(f"Epoch {epoch} terminated.")

except KeyboardInterrupt:
    shared_critic.save_weights(path_critic + "shared_critic_exp_3.weights.h5")
    shared_actor.save_weights(path_actor + "shared_actor_exp_3.weights.h5")
    print("User interrupted training. Saving weights")

Episode [  1] terminated at timestep 400. cumulative reward:   3. avg reward: 3.0
Performing stocastic gradient descent with 4 epochs.
Epoch 1 terminated.
Epoch 2 terminated.
Epoch 3 terminated.
Epoch 4 terminated.
Episode [  2] terminated at timestep 400. cumulative reward:   0. avg reward: 1.5
Performing stocastic gradient descent with 4 epochs.
Epoch 1 terminated.
Epoch 2 terminated.
Epoch 3 terminated.
Epoch 4 terminated.
Episode [  3] terminated at timestep 400. cumulative reward:   8. avg reward: 3.667
Performing stocastic gradient descent with 4 epochs.
Epoch 1 terminated.
Epoch 2 terminated.
Epoch 3 terminated.
Epoch 4 terminated.
Episode [  4] terminated at timestep 400. cumulative reward:  14. avg reward: 6.25
Performing stocastic gradient descent with 4 epochs.
Epoch 1 terminated.
Epoch 2 terminated.
Epoch 3 terminated.
Epoch 4 terminated.
Episode [  5] terminated at timestep 400. cumulative reward:   6. avg reward: 6.2
Performing stocastic gradient descent with 4 epochs.
Ep

In [37]:
# Here we create an evaluator for the cramped_room layout
layout = "cramped_room"
ae = AgentEvaluator.from_layout_name(mdp_params={"layout_name": layout, "old_dynamics": True},
                                     env_params={"horizon": 400})

ap = AgentPair(agent_1, agent_2)

trajs = ae.evaluate_agent_pair(ap, 10)
# trajs = ae.evaluate_human_model_pair(1)

StateVisualizer().display_rendered_trajectory(trajs, ipython_display=True)

Avg rew: 0.00 (std: 0.00, se: 0.00); avg len: 400.00; : 100%|██████████| 10/10 [00:57<00:00,  5.74s/it]


interactive(children=(IntSlider(value=0, description='timestep', max=399), Output()), _dom_classes=('widget-in…

# Trial with batches and past reward.

In [6]:
number_of_frames = 400
layout_name = "cramped_room"
base_mdp = OvercookedGridworld.from_layout_name(layout_name=layout_name) #or other layout
base_env = OvercookedEnv.from_mdp(base_mdp, info_level=0, horizon=number_of_frames)
env = Overcooked(base_env=base_env, featurize_fn=base_env.featurize_state_mdp)

In [7]:
alpha_w = 1e-5
alpha_t = 1e-6
critic_optimizer = Adam(learning_rate=alpha_w)
actor_optimizer = Adam(learning_rate=alpha_t)

In [8]:
input_shape = env.observation_space._shape

shared_actor = Policy(
    input_shape=input_shape,
    num_actions=Action.NUM_ACTIONS,
    optimizer=actor_optimizer
    )

shared_critic = ValueFunctionApproximator(
    input_shape=input_shape,
    optimizer=critic_optimizer
    )

In [9]:
agent_1 = MyAgent(
    actor=shared_actor,
    old_policy=None,
    critic=shared_critic,
    idx=0,
    base_env=base_env,
)
agent_2 = MyAgent(
    actor=shared_actor,
    old_policy=None,
    critic=shared_critic,
    idx=1,
    base_env=base_env,
)

In [14]:
shared_actor.load_weights(path_actor + "shared_actor_exp_7.weights.h5")
shared_critic.load_weights(path_critic + "shared_critic_exp_7.weights.h5")

In [20]:
number_of_episodes = 1000
number_of_epochs = 2 # prima era 4
batch_size = 20
average_reward = 0
best_avg = 0
previous_action_to_reward = 5
gamma = 0.95

try:
    for episode in range(1, number_of_episodes + 1):
        actions = []
        deltas = []
        observations = []

        t = 0
        obs = env.reset()
        obs = obs['both_agent_obs'] 
        
        done = False
        cumulative_reward = 0

        # if (episode) % 50 == 0:
        #     shared_critic.save_weights(path_critic + "shared_critic_exp_6.weights.h5")
        #     shared_actor.save_weights(path_actor + "shared_actor_exp_6.weights.h5")

        while not done:
            action1 = agent_1.action(obs)
            action2 = agent_2.action(obs)
            player_1_action = Action.ACTION_TO_INDEX[action1[0]]
            player_2_action = Action.ACTION_TO_INDEX[action2[0]]
            action = (player_1_action, player_2_action)

            actions.append(action)
            observations.append(obs)

            new_obs, reward, done, env_info = env.step(action)
            shaped_reward = sum(env_info['shaped_r_by_agent']) # let's use shaped reward for learning how to play first.

            new_obs = new_obs['both_agent_obs']

            # if shaped_reward != 0:
            #     shaped_reward = shaped_reward * shaped_reward_factor # discounting the shaped reward
            #     shaped_reward_factor *= discount_rate

            total_reward = reward + shaped_reward

            cumulative_reward += total_reward

            if total_reward > 0:
                previous_action_to_reward = 5
                if t > previous_action_to_reward:
                    for i in range(t-1, t-previous_action_to_reward-1, -1):
                        deltas[i] += total_reward
                else:
                    for i in range(t-1,-1,-1):
                        deltas[i] += total_reward

            # compute delta = R + v^(St+1) - v^(St) where v^(St+1) = 0 if done
            if done:
                delta = total_reward - shared_critic.call(obs)
            else:
                delta = total_reward + gamma*shared_critic.call(new_obs) - shared_critic.call(obs)

            deltas.append(delta)
            
            # update state (obs = new_obs)
            obs = new_obs

            # think about training the critic by itself for a while
            t += 1

        average_reward = 1/(episode)*( cumulative_reward + (episode-1)*average_reward)

        if average_reward > best_avg:
            best_avg = average_reward
            shared_critic.save_weights(path_critic + "shared_critic_exp_7.weights.h5")
            shared_actor.save_weights(path_actor + "shared_actor_exp_7.weights.h5")

        print(f"Episode [{episode:>3d}] terminated at timestep {t}. cumulative reward: {cumulative_reward:>3d}. avg reward: {round(average_reward, 3)}")
        print(f"Performing stocastic gradient descent with {number_of_epochs} epochs.")
        for epoch in range(1, number_of_epochs + 1):
            num_batches = len(actions) // batch_size
            shuffled_indices = tf.random.shuffle(tf.range(len(actions)))
            for batch in range(num_batches):
                if batch == num_batches: # last batch
                    idx = shuffled_indices[batch*batch_size:]
                else:
                    idx = shuffled_indices[batch*batch_size:(batch+1)*batch_size]

                deltas_batch = tf.squeeze(tf.gather(deltas, idx), axis=-1)
                actions_batch = tf.gather(actions, idx)
                observations_batch = tf.gather(observations, idx)

                shared_critic.train_batch(deltas_batch, observations_batch)
                shared_actor.train_batch(deltas_batch, observations_batch, actions_batch)
            print(f"Epoch {epoch} terminated.")

except KeyboardInterrupt:
    shared_critic.save_weights(path_critic + "shared_critic_exp_7.weights.h5")
    shared_actor.save_weights(path_actor + "shared_actor_exp_7.weights.h5")
    print("User interrupted training. Saving weights")

Episode [  1] terminated at timestep 400. cumulative reward:  28. avg reward: 28.0
Performing stocastic gradient descent with 2 epochs.
Epoch 1 terminated.
Epoch 2 terminated.
Episode [  2] terminated at timestep 400. cumulative reward:  23. avg reward: 25.5
Performing stocastic gradient descent with 2 epochs.
Epoch 1 terminated.
Epoch 2 terminated.
Episode [  3] terminated at timestep 400. cumulative reward:  25. avg reward: 25.333
Performing stocastic gradient descent with 2 epochs.
Epoch 1 terminated.
Epoch 2 terminated.
Episode [  4] terminated at timestep 400. cumulative reward:  36. avg reward: 28.0
Performing stocastic gradient descent with 2 epochs.
Epoch 1 terminated.
Epoch 2 terminated.
Episode [  5] terminated at timestep 400. cumulative reward:  14. avg reward: 25.2
Performing stocastic gradient descent with 2 epochs.
Epoch 1 terminated.
Epoch 2 terminated.
Episode [  6] terminated at timestep 400. cumulative reward:  22. avg reward: 24.667
Performing stocastic gradient des

In [21]:
# Here we create an evaluator for the cramped_room layout
layout = "cramped_room"
ae = AgentEvaluator.from_layout_name(mdp_params={"layout_name": layout, "old_dynamics": True},
                                     env_params={"horizon": 400})

ap = AgentPair(agent_1, agent_2)

trajs = ae.evaluate_agent_pair(ap, 10)
# trajs = ae.evaluate_human_model_pair(1)

StateVisualizer().display_rendered_trajectory(trajs, ipython_display=True)

Avg rew: 30.00 (std: 10.00, se: 3.16); avg len: 400.00; : 100%|██████████| 10/10 [00:48<00:00,  4.84s/it]


interactive(children=(IntSlider(value=0, description='timestep', max=399), Output()), _dom_classes=('widget-in…

# Trial with ordered batches

In [38]:
number_of_frames = 400
layout_name = "cramped_room"
base_mdp = OvercookedGridworld.from_layout_name(layout_name=layout_name) #or other layout
base_env = OvercookedEnv.from_mdp(base_mdp, info_level=0, horizon=number_of_frames)
env = Overcooked(base_env=base_env, featurize_fn=base_env.featurize_state_mdp)

In [39]:
alpha_w = 1e-5
alpha_t = 1e-6
critic_optimizer = Adam(learning_rate=alpha_w)
actor_optimizer = Adam(learning_rate=alpha_t)

In [40]:
input_shape = env.observation_space._shape

shared_actor = Policy(
    input_shape=input_shape,
    num_actions=Action.NUM_ACTIONS,
    optimizer=actor_optimizer
    )

shared_critic = ValueFunctionApproximator(
    input_shape=input_shape,
    optimizer=critic_optimizer
    )

In [41]:
agent_1 = MyAgent(
    actor=shared_actor,
    critic=shared_critic,
    idx=0,
    base_env=base_env,
)
agent_2 = MyAgent(
    actor=shared_actor,
    critic=shared_critic,
    idx=1,
    base_env=base_env,
)

In [42]:
number_of_episodes = 300
number_of_epochs = 4
batch_size = 20
average_reward = 0
gamma = 0.95

try:
    for episode in range(1, number_of_episodes + 1):
        actions = []
        deltas = []
        observations = []

        t = 0
        obs = env.reset()
        
        done = False
        cumulative_reward = 0

        if (episode) % 50 == 0:
            shared_critic.save_weights(path_critic + "shared_critic_exp_4.weights.h5")
            shared_actor.save_weights(path_actor + "shared_actor_exp_4.weights.h5")

        while not done:
            action1 = agent_1.action(obs['both_agent_obs'])
            action2 = agent_2.action(obs['both_agent_obs'])
            player_1_action = Action.ACTION_TO_INDEX[action1[0]]
            player_2_action = Action.ACTION_TO_INDEX[action2[0]]

            if obs['other_agent_env_idx'] == 0:
                my_obs = (obs['both_agent_obs'][1],obs['both_agent_obs'][0])
                obs['both_agent_obs'] = my_obs # BRO I WAS TOO LAZY TO OVERRIDE OBS. DON'T FORGET IT
                action = (player_2_action, player_1_action)
            else:
                action = (player_1_action, player_2_action)

            actions.append(action)
            observations.append(obs['both_agent_obs'])

            new_obs, reward, done, env_info = env.step(action)
            shaped_reward = sum(env_info['shaped_r_by_agent']) # let's use shaped reward for learning how to play first.

            if shaped_reward != 0:
                shaped_reward = shaped_reward * shaped_reward_factor # discounting the shaped reward
                shaped_reward_factor *= discount_rate

            total_reward = reward + shaped_reward

            cumulative_reward += total_reward

            # compute delta = R + v^(St+1) - v^(St) where v^(St+1) = 0 if done
            if done:
                delta = total_reward - shared_critic.call(obs['both_agent_obs'])
            else:
                delta = total_reward + gamma*shared_critic.call(new_obs['both_agent_obs']) - shared_critic.call(obs['both_agent_obs'])

            deltas.append(delta)
            
            # update state (obs = new_obs)
            obs = new_obs

            # think about training the critic by itself for a while
            t += 1

        average_reward = 1/(episode)*( cumulative_reward + (episode-1)*average_reward)
        print(f"Episode [{episode:>3d}] terminated at timestep {t}. cumulative reward: {cumulative_reward:>3d}. avg reward: {round(average_reward, 3)}")
        print(f"Performing stocastic gradient descent with {number_of_epochs} epochs.")
        for epoch in range(1, number_of_epochs + 1):
            num_batches = len(actions) // batch_size
            shuffled_indices = tf.random.shuffle(tf.range(len(actions)))
            for batch in range(num_batches):
                if batch == num_batches: # last batch
                    idx = shuffled_indices[batch*batch_size:]
                else:
                    idx = shuffled_indices[batch*batch_size:(batch+1)*batch_size]

                deltas_batch = tf.squeeze(tf.gather(deltas, idx), axis=-1)
                actions_batch = tf.gather(actions, idx)
                observations_batch = tf.gather(observations, idx)

                shared_critic.train_batch(deltas_batch, observations_batch)
                shared_actor.train_batch(deltas_batch, observations_batch, actions_batch)
            print(f"Epoch {epoch} terminated.")

except KeyboardInterrupt:
    shared_critic.save_weights(path_critic + "shared_critic_exp_4.weights.h5")
    shared_actor.save_weights(path_actor + "shared_actor_exp_4.weights.h5")
    print("User interrupted training. Saving weights")

Episode [  1] terminated at timestep 400. cumulative reward:   9. avg reward: 9.0
Performing stocastic gradient descent with 4 epochs.
Epoch 1 terminated.
Epoch 2 terminated.
Epoch 3 terminated.
Epoch 4 terminated.
Episode [  2] terminated at timestep 400. cumulative reward:   9. avg reward: 9.0
Performing stocastic gradient descent with 4 epochs.
Epoch 1 terminated.
Epoch 2 terminated.
Epoch 3 terminated.
Epoch 4 terminated.
Episode [  3] terminated at timestep 400. cumulative reward:   3. avg reward: 7.0
Performing stocastic gradient descent with 4 epochs.
Epoch 1 terminated.
Epoch 2 terminated.
Epoch 3 terminated.
Epoch 4 terminated.
Episode [  4] terminated at timestep 400. cumulative reward:  11. avg reward: 8.0
Performing stocastic gradient descent with 4 epochs.
Epoch 1 terminated.
Epoch 2 terminated.
Epoch 3 terminated.
Epoch 4 terminated.
Episode [  5] terminated at timestep 400. cumulative reward:   3. avg reward: 7.0
Performing stocastic gradient descent with 4 epochs.
Epoch

In [ ]:
# Here we create an evaluator for the cramped_room layout
layout = "cramped_room"
ae = AgentEvaluator.from_layout_name(mdp_params={"layout_name": layout, "old_dynamics": True},
                                     env_params={"horizon": 400})

ap = AgentPair(agent_1, agent_2)

trajs = ae.evaluate_agent_pair(ap, 10)
# trajs = ae.evaluate_human_model_pair(1)

StateVisualizer().display_rendered_trajectory(trajs, ipython_display=True)

# Trial with PPO

In [84]:
number_of_frames = 400
layout_name = "cramped_room"
base_mdp = OvercookedGridworld.from_layout_name(layout_name=layout_name) #or other layout
base_env = OvercookedEnv.from_mdp(base_mdp, info_level=0, horizon=number_of_frames)
env = Overcooked(base_env=base_env, featurize_fn=base_env.featurize_state_mdp)

In [85]:
alpha_w = 1e-5
alpha_t = 1e-6
critic_optimizer = Adam(learning_rate=alpha_w)
actor_optimizer = Adam(learning_rate=alpha_t)

In [86]:
input_shape = env.observation_space._shape

shared_actor = Policy(
    input_shape=input_shape,
    num_actions=Action.NUM_ACTIONS,
    optimizer=actor_optimizer
    )

shared_old_policy = Policy(
    input_shape=input_shape,
    num_actions=Action.NUM_ACTIONS,
    optimizer=None
    )

shared_critic = ValueFunctionApproximator(
    input_shape=input_shape,
    optimizer=critic_optimizer
    )

In [87]:
agent_1 = MyAgent(
    actor=shared_actor,
    old_policy=shared_old_policy,
    critic=shared_critic,
    idx=0,
    base_env=base_env,
)
agent_2 = MyAgent(
    actor=shared_actor,
    old_policy=shared_old_policy,
    critic=shared_critic,
    idx=1,
    base_env=base_env,
)

In [88]:
agent_1.update_old_policy()

In [89]:
number_of_episodes = 500
number_of_epochs = 4
batch_size = 20
average_reward = 0
gamma = 0.95

try:
    for episode in range(1, number_of_episodes + 1):
        actions = []
        deltas = []
        observations = []

        t = 0
        obs = env.reset()
        obs = obs['both_agent_obs'] 
        
        done = False
        cumulative_reward = 0

        if (episode) % 50 == 0:
            shared_critic.save_weights(path_critic + "shared_critic_exp_5.weights.h5")
            shared_actor.save_weights(path_actor + "shared_actor_exp_5.weights.h5")

        while not done:
            action1 = agent_1.action(obs)
            action2 = agent_2.action(obs)
            player_1_action = Action.ACTION_TO_INDEX[action1[0]]
            player_2_action = Action.ACTION_TO_INDEX[action2[0]]
            action = (player_1_action, player_2_action)

            actions.append(action)
            observations.append(obs)

            new_obs, reward, done, env_info = env.step(action)
            shaped_reward = sum(env_info['shaped_r_by_agent']) # let's use shaped reward for learning how to play first.

            new_obs = new_obs['both_agent_obs']

            if shaped_reward != 0:
                shaped_reward = shaped_reward * shaped_reward_factor # discounting the shaped reward
                shaped_reward_factor *= discount_rate

            total_reward = reward + shaped_reward

            cumulative_reward += total_reward

            # compute delta = R + v^(St+1) - v^(St) where v^(St+1) = 0 if done
            if done:
                delta = total_reward - shared_critic.call(obs)
            else:
                delta = total_reward + gamma*shared_critic.call(new_obs) - shared_critic.call(obs)

            deltas.append(delta)
            
            # update state (obs = new_obs)
            obs = new_obs

            # think about training the critic by itself for a while
            t += 1

        average_reward = 1/(episode)*( cumulative_reward + (episode-1)*average_reward)
        print(f"Episode [{episode:>3d}] terminated at timestep {t}. cumulative reward: {cumulative_reward:>3d}. avg reward: {round(average_reward, 3)}")
        print(f"Performing stocastic gradient descent with {number_of_epochs} epochs.")
        for epoch in range(1, number_of_epochs + 1):
            num_batches = len(actions) // batch_size
            shuffled_indices = tf.random.shuffle(tf.range(len(actions)))
            for batch in range(num_batches):
                if batch == num_batches: # last batch
                    idx = shuffled_indices[batch*batch_size:]
                else:
                    idx = shuffled_indices[batch*batch_size:(batch+1)*batch_size]

                deltas_batch = tf.squeeze(tf.gather(deltas, idx), axis=-1)
                actions_batch = tf.gather(actions, idx)
                observations_batch = tf.gather(observations, idx)

                shared_critic.train_batch(deltas_batch, observations_batch)
                shared_actor.train_batch_PPO(deltas_batch, observations_batch, actions_batch, agent_1.old_policy)
            print(f"Epoch {epoch} terminated.")
        agent_1.update_old_policy()

except KeyboardInterrupt:
    shared_critic.save_weights(path_critic + "shared_critic_exp_5.weights.h5")
    shared_actor.save_weights(path_actor + "shared_actor_exp_5.weights.h5")
    print("User interrupted training. Saving weights")

Episode [  1] terminated at timestep 400. cumulative reward:   3. avg reward: 3.0
Performing stocastic gradient descent with 4 epochs.
Epoch 1 terminated.
Epoch 2 terminated.
Epoch 3 terminated.
Epoch 4 terminated.
Episode [  2] terminated at timestep 400. cumulative reward:   3. avg reward: 3.0
Performing stocastic gradient descent with 4 epochs.
Epoch 1 terminated.
Epoch 2 terminated.
Epoch 3 terminated.
Epoch 4 terminated.
Episode [  3] terminated at timestep 400. cumulative reward:   3. avg reward: 3.0
Performing stocastic gradient descent with 4 epochs.
Epoch 1 terminated.
Epoch 2 terminated.
Epoch 3 terminated.
Epoch 4 terminated.
Episode [  4] terminated at timestep 400. cumulative reward:   8. avg reward: 4.25
Performing stocastic gradient descent with 4 epochs.
Epoch 1 terminated.
Epoch 2 terminated.
Epoch 3 terminated.
Epoch 4 terminated.
Episode [  5] terminated at timestep 400. cumulative reward:   3. avg reward: 4.0
Performing stocastic gradient descent with 4 epochs.
Epoc

In [ ]:
# Here we create an evaluator for the cramped_room layout
layout = "cramped_room"
ae = AgentEvaluator.from_layout_name(mdp_params={"layout_name": layout, "old_dynamics": True},
                                     env_params={"horizon": 400})

ap = AgentPair(agent_1, agent_2)

trajs = ae.evaluate_agent_pair(ap, 10)
# trajs = ae.evaluate_human_model_pair(1)

StateVisualizer().display_rendered_trajectory(trajs, ipython_display=True)

Avg rew: 0.00 (std: 0.00, se: 0.00); avg len: 400.00; : 100%|██████████| 10/10 [00:57<00:00,  5.74s/it]


interactive(children=(IntSlider(value=0, description='timestep', max=399), Output()), _dom_classes=('widget-in…

# Using trajectories
 When evaluating trajectory of my agents, the agents are fed with the overcooked state.  
 Since my agent use the "both_agent_obs", i can convert the overcooked state in "both_agent_obs" through base_env.featurize_state_mdp(state).  
 The problem here is that i need to pay attention to the index since this method returns always in the POV of agent 0.  
 What i can do is to retrieve the index with env.agent_idx and i will have the following equality:  
 base_env.featurize_state_mdp(state)[0] == obs['both_agent_obs'][env.agent_idx]  
 I need to find a way to include the environment on my agent class and be able to call the agent_idx in order to preprocess the state and use the right order.
 During testing this should not be a huge problem since our network should be able to generalize the position... let's do without it!

In [32]:
obs = env.reset()
state = obs['overcooked_state']
transformed_obs = base_env.featurize_state_mdp(state)
print(env.agent_idx)
transformed_obs[env.agent_idx] == obs['both_agent_obs'][0]


0


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

# Let's try implementing training with batches
The idea is to compute some steps in the episode (let's say the whole episode), then you perform stochastic gradient descent with batches. the batch is the whole episode, and you train on minibatches of the episode, performing multiple epochs (let's say 10.)  
`Note:` THIS IS STILL ACTOR CRITIC KINDA

In [40]:
class PolicyBatch(Model):
    def __init__(self, input_shape, num_actions, optimizer):
        super().__init__()
        self.input_shape = input_shape
        self.num_actions = num_actions
        self.optimizer = optimizer
        self.input_a = Input(shape=(self.input_shape))
        self.input_b = Input(shape=(self.input_shape))
        self.dense_1 = layers.Dense(128, activation='tanh')
        self.dense_2 = layers.Dense(256, activation='tanh')
        self.dense_3 = layers.Dense(128, activation='tanh')
        self.policy_a = layers.Dense(self.num_actions, activation='softmax', name="policy_a")
        self.policy_b = layers.Dense(self.num_actions, activation='softmax', name="policy_b")
        self.build_model()

    def preprocess(self, obs_batch):
        if isinstance(obs_batch, Tuple):
            obs_batch = [obs_batch] # to handle the case where obs_batch is a single observation

        obs_1, obs_2 = zip(*obs_batch)
        obs_batch = tf.concat([tf.stack(obs_1), tf.stack(obs_2)], axis=-1)
        return obs_batch


    def call(self, obs_batch, training=False):
        x = self.preprocess(obs_batch)
        x = self.dense_1(x)
        x = self.dense_2(x)
        x = self.dense_3(x)
        policy_a = self.policy_a(x)
        policy_b = self.policy_b(x)
        return (policy_a, policy_b) # self.call(obs_batch)[0][N]: n-th policy of agent 1

    def build_model(self):
        # computing a forward pass in order to automatically build the model
        dummy_input = (
            tf.zeros((1, 96)),
            tf.zeros((1, 96))
        )
        _ = self(dummy_input)

    def train_step(self, deltas_batch: tf.Tensor, obs_batch, actions_batch):
        # update t with t + alpha_t*delta*grad_pi^(A|S) where A is the action taken before reaching St+1
        with tf.GradientTape() as tape:
            pi = self.call(obs_batch, training=True)
            log_pi = tf.math.log(pi)
            # processed_log_pi = -deltas_batch * log_pi
            # pi_a = 0
            # for i in range(len(actions_batch)):
            #     pi_a += processed_log_pi[0][i][actions_batch[i][0]] + processed_log_pi[1][i][actions_batch[i][1]]
            pi_a1 = tf.gather(log_pi[0], actions_batch[:, 0], axis=1, batch_dims=1)
            pi_a2 = tf.gather(log_pi[1], actions_batch[:, 1], axis=1, batch_dims=1)

            # Now compute the weighted sum over the batch
            pi_a = -tf.reduce_sum(tf.squeeze(deltas_batch) * (pi_a1 + pi_a2))


        grad_pi_a = tape.gradient(pi_a, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grad_pi_a, self.trainable_weights))

        # # Assuming shape of log_pi: (2, batch_size, num_actions)
        # # actions_batch: (batch_size, 2), each row is [a1, a2]
        # pi_a1 = tf.gather(log_pi[0], actions_batch[:, 0], axis=1, batch_dims=1)
        # pi_a2 = tf.gather(log_pi[1], actions_batch[:, 1], axis=1, batch_dims=1)

        # # Now compute the weighted sum over the batch
        # loss = -tf.reduce_sum(tf.squeeze(deltas_batch) * (pi_a1 + pi_a2))



class ValueFunctionApproximatorBatch(Model):
    def __init__(self, input_shape, optimizer):
        super().__init__()
        self.input_shape = input_shape
        self.optimizer = optimizer
        self.input_a = Input(shape=(self.input_shape))
        self.input_b = Input(shape=(self.input_shape))
        self.dense_1 = layers.Dense(128, activation='tanh')
        self.dense_2 = layers.Dense(256, activation='tanh')
        self.dense_3 = layers.Dense(128, activation='tanh')
        # self.value_function = layers.Dense(1, activation='relu', name="value_function")
        self.value_function = layers.Dense(1, name="value_function")
        self.build_model()

    def preprocess(self, obs_batch):
        if isinstance(obs_batch, Tuple):
            obs_batch = [obs_batch] # to handle the case where obs_batch is a single observation

        obs_1, obs_2 = zip(*obs_batch)
        obs_batch = tf.concat([tf.stack(obs_1), tf.stack(obs_2)], axis=-1)
        return obs_batch


    def call(self, obs_batch, training=False):
        x = self.preprocess(obs_batch)
        x = self.dense_1(x)
        x = self.dense_2(x)
        x = self.dense_3(x)
        value_function = self.value_function(x)
        return value_function

    def build_model(self):
        # computing a forward pass in order to automatically build the model
        dummy_input =  (
            tf.zeros((1, 96)),
            tf.zeros((1, 96))
        )
        _ = self(dummy_input)

    def train_step(self, deltas_batch: tf.Tensor, obs_batch): # deltas is a tf.Tensor of shape (batch_size,1)
        # update w with w + alpha_w*grad_v^(St)*delta
        with tf.GradientTape() as tape:
            state_value = self.call(obs_batch, training=True)
            loss = tf.reduce_mean(tf.square(deltas_batch)) # adding a term that minimizes the distance between the true and the estimated value
            processed_state_value = -deltas_batch * state_value + loss

        grad_state_value = tape.gradient(processed_state_value, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grad_state_value, self.trainable_weights))

class MyAgentBatch(Agent):
    """
    This class is more a couple of actors since we use shared networks and the output are 2!!!
    For now let's treat it like a single player identified by self.index
    """
    def __init__(self, actor, critic, idx, base_env: OvercookedEnv):
        super().__init__()
        self.actor = actor
        self.critic = critic
        self.idx = idx
        if not self.idx in [0,1]:
            raise AssertionError("The index of the agent must be either 0 or 1!")
        self.base_env = base_env

    def action(self, obs):
        """
        obs: preprocessed observation (or overcookedstate)
        We want to output the action given the state. can use a NN!
        should return a tuple (Action, Dict)
        Dict should contain info about the action ('action_probs': numpy array)
        """
        if isinstance(obs, OvercookedState):
            # this is useful for translating the OvercookedState
            # into observation that can be fed into the NN.
            obs_from_state = self.base_env.featurize_state_mdp(obs)
            obs = (obs_from_state[0],obs_from_state[1])


        action_probs = self.actor.call(obs)[self.idx].numpy()
        # if np.random.random() > self.epsilon:
        #     action = Action.argmax(action_probs) # greedy selection
        # else:
        #     action_idx = np.random.choice(range(0,6), size=1)[0]
        #     action = Action.INDEX_TO_ACTION[action_idx] # random exploration
        sample = tf.random.categorical(tf.math.log(action_probs), num_samples = 1)
        action_idx = tf.squeeze(sample, axis=-1)[0].numpy()
        action = Action.INDEX_TO_ACTION[action_idx]
        return (action, {'action_probs': action_probs})

    def actions(self, obss):
        """
        Look at the documentation of the Agent class
        """
        pass

    def update(self, obs, reward):
        """
        What do we need to update?
        """
        pass

    # def decay_epsilon(self, decay):
    #     if self.epsilon - decay <= 0.5:
    #         self.epsilon = 0.5
    #     else:
    #         self.epsilon -= decay

In [41]:
number_of_frames = 400
layout_name = "cramped_room"
base_mdp = OvercookedGridworld.from_layout_name(layout_name=layout_name) #or other layout
base_env = OvercookedEnv.from_mdp(base_mdp, info_level=0, horizon=number_of_frames)
env = Overcooked(base_env=base_env, featurize_fn=base_env.featurize_state_mdp)

alpha_t = 1e-6
alpha_w = 1e-5

optimizer_actor = Adam(learning_rate=alpha_t)
optimizer_critic = Adam(learning_rate=alpha_w)

input_shape = env.observation_space._shape

shared_actor_batch = PolicyBatch(
    input_shape=input_shape,
    num_actions=Action.NUM_ACTIONS,
    optimizer=optimizer_actor
)

shared_critic_batch = ValueFunctionApproximatorBatch(
    input_shape=input_shape,
    optimizer=optimizer_critic
    )

agent_1 = MyAgentBatch(
    actor=shared_actor_batch,
    critic=shared_critic_batch,
    idx=0,
    base_env=base_env
)
agent_2 = MyAgentBatch(
    actor=shared_actor_batch,
    critic=shared_critic_batch,
    idx=1,
    base_env=base_env
)

In [42]:
path_critic = "networks/critic/"
path_actor = "networks/actor/"

In [43]:
number_of_episodes = 1
batch_size = 30
number_of_epochs = 2
average_reward = 0
try:
    for episode in range(1, number_of_episodes + 1):
        timestep = 0
        obs = env.reset()
        done = False

        # shaped_reward_factor = 1 # multiplicative factor for discounting shaped reward.
        # discount_rate = 1 # will decrease the shaped reward every time
        cumulative_reward = 0

        if (episode) % 75 == 0:
            shared_critic_batch.save_weights(path_critic + "shared_critic_batch.weights.h5")
            shared_actor_batch.save_weights(path_actor + "shared_actor_batch.weights.h5")

        actions = []
        deltas = []
        observations = []
        rewards = []

        while not done:
            # reordering the actions based on the POV of the environment
            if obs['other_agent_env_idx'] == 0:
                my_obs = (obs['both_agent_obs'][1],obs['both_agent_obs'][0])
                obs['both_agent_obs'] = my_obs

            action1 = agent_1.action(obs['both_agent_obs'])
            action2 = agent_2.action(obs['both_agent_obs'])
            player_1_action = Action.ACTION_TO_INDEX[action1[0]]
            player_2_action = Action.ACTION_TO_INDEX[action2[0]]
            
            if obs['other_agent_env_idx'] == 1:
                action = (player_1_action, player_2_action)
                # action = (agent_1_actions[timestep], agent_2_actions[timestep])
            else:
                action = (player_2_action,player_1_action)
                # action = (agent_2_actions[timestep], agent_1_actions[timestep])

            actions.append(action)
            observations.append(obs['both_agent_obs'])

            new_obs, reward, done, env_info = env.step(action)
            shaped_reward = sum(env_info['shaped_r_by_agent']) # let's use shaped reward for learning how to play first.

            # if shaped_reward != 0:
            #     shaped_reward = shaped_reward * shaped_reward_factor # discounting the shaped reward
            #     shaped_reward_factor *= discount_rate


            total_reward = reward + shaped_reward
            rewards.append(total_reward)
            cumulative_reward += total_reward

            # compute delta = R + v^(St+1) - v^(St) where v^(St+1) = 0 if done
            # if done:
            #     delta = total_reward - shared_critic_batch.call(obs['both_agent_obs'])
            # else:
            #     delta = total_reward + shared_critic_batch.call(new_obs['both_agent_obs']) - shared_critic_batch.call(obs['both_agent_obs'])

            # deltas.append(delta)

            # update state (obs = new_obs)
            obs = new_obs

            timestep += 1

        average_reward = 1/(episode)*( cumulative_reward + (episode-1)*average_reward)
        print(f"Episode [{episode:>3d}] terminated at timestep {timestep}. cumulative reward: {cumulative_reward:>3d}. avg reward: {round(average_reward, 3)}.")

        print(f"Computing the deltas (MONTE-CARLO)")
        for t in range(len(actions)):
            G_t = np.sum(rewards[t:])
            delta = G_t - shared_critic_batch.call(observations[t])
            deltas.append(delta)

        print(f"Performing stocastic gradient descent with {number_of_epochs} epochs.")
        for epoch in range(1, number_of_epochs + 1):
            num_batches = len(actions) // batch_size
            shuffled_indices = tf.random.shuffle(tf.range(len(actions)))
            for batch in range(num_batches):
                if batch == num_batches: # last batch
                    idx = shuffled_indices[batch*batch_size:]
                else:
                    idx = shuffled_indices[batch*batch_size:(batch+1)*batch_size]

                deltas_batch = tf.squeeze(tf.gather(deltas, idx), axis=-1)
                actions_batch = tf.gather(actions, idx)
                observations_batch = tf.gather(observations, idx)

                shared_critic_batch.train_step(deltas_batch, observations_batch)
                shared_actor_batch.train_step(deltas_batch, observations_batch, actions_batch)
            print(f"Epoch {epoch} terminated.")



        # shared_critic.save_weights(path_critic + "shared_critic.weights.h5")
        # shared_actor.save_weights(path_actor + "shared_actor.weights.h5")
except:
    shared_critic_batch.save_weights(path_critic + "shared_critic_batch.weights.h5")
    shared_actor_batch.save_weights(path_actor + "shared_actor_batch.weights.h5")
    print("Unexpected interruption of training. Saving weights")

Episode [  1] terminated at timestep 400. cumulative reward:   3. avg reward: 3.0.
Computing the deltas (MONTE-CARLO)
Performing stocastic gradient descent with 2 epochs.
Epoch 1 terminated.
Epoch 2 terminated.


In [127]:
# Here we create an evaluator for the cramped_room layout
layout = "cramped_room"
ae = AgentEvaluator.from_layout_name(mdp_params={"layout_name": layout, "old_dynamics": True},
                                     env_params={"horizon": 400})

ap = AgentPair(agent_1, agent_2)

trajs = ae.evaluate_agent_pair(ap, 10)

StateVisualizer().display_rendered_trajectory(trajs, ipython_display=True)

  0%|          | 0/10 [00:00<?, ?it/s]

Avg rew: 0.00 (std: 0.00, se: 0.00); avg len: 400.00; : 100%|██████████| 10/10 [00:47<00:00,  4.73s/it]


interactive(children=(IntSlider(value=0, description='timestep', max=399), Output()), _dom_classes=('widget-in…

In [129]:
shared_actor_batch(observations[-1])

(<tf.Tensor: shape=(1, 6), dtype=float32, numpy=
 array([[0.22263195, 0.1434645 , 0.07354847, 0.18801843, 0.34869647,
         0.02364018]], dtype=float32)>,
 <tf.Tensor: shape=(1, 6), dtype=float32, numpy=
 array([[0.15698901, 0.10884099, 0.09701831, 0.19173887, 0.3188748 ,
         0.12653792]], dtype=float32)>)

In [60]:
# Here we create an evaluator for the cramped_room layout
layout = "cramped_room"
ae = AgentEvaluator.from_layout_name(mdp_params={"layout_name": layout, "old_dynamics": True},
                                     env_params={"horizon": 400})

ap = AgentPair(RandomAgent(all_actions=True), RandomAgent(all_actions=True))

trajs = ae.evaluate_agent_pair(ap, 10)

StateVisualizer().display_rendered_trajectory(trajs, ipython_display=True)

Avg rew: 0.00 (std: 0.00, se: 0.00); avg len: 400.00; : 100%|██████████| 10/10 [00:00<00:00, 12.70it/s]


interactive(children=(IntSlider(value=0, description='timestep', max=399), Output()), _dom_classes=('widget-in…

# TRIALS

actor batch

# Let's use different NNs for the agents:

In [ ]:
class SinglePolicyBatch(Model):
    def __init__(self, input_shape, num_actions, optimizer):
        super().__init__()
        self.input_shape = input_shape
        self.num_actions = num_actions
        self.optimizer = optimizer
        self.input_a = Input(shape=(self.input_shape))
        self.input_b = Input(shape=(self.input_shape))
        self.dense_1 = layers.Dense(128, activation='tanh')
        self.dense_2 = layers.Dense(256, activation='tanh')
        self.dense_3 = layers.Dense(128, activation='tanh')
        self.policy = layers.Dense(self.num_actions, activation='softmax', name="policy")
        self.build_model()

    def preprocess(self, obs_batch):
        if isinstance(obs_batch, Tuple):
            obs_batch = [obs_batch] # to handle the case where obs_batch is a single observation

        obs_1, obs_2 = zip(*obs_batch)
        obs_batch = tf.concat([tf.stack(obs_1), tf.stack(obs_2)], axis=-1)
        return obs_batch


    def call(self, obs_batch, training=False):
        x = self.preprocess(obs_batch)
        x = self.dense_1(x)
        x = self.dense_2(x)
        x = self.dense_3(x)
        policy = self.policy(x)
        return policy

    def build_model(self):
        # computing a forward pass in order to automatically build the model
        dummy_input = (
            tf.zeros((1, 96)),
            tf.zeros((1, 96))
        )
        _ = self(dummy_input)

    def train_step(self, deltas_batch: tf.Tensor, obs_batch, actions_batch):
        # update t with t + alpha_t*delta*grad_pi^(A|S) where A is the action taken before reaching St+1
        with tf.GradientTape() as tape:
            pi = self.call(obs_batch, training=True)
            log_pi = tf.math.log(pi)
            processed_log_pi = -deltas_batch * log_pi
            pi_a = 0
            for i in range(len(actions_batch)):
                # THIS NEED TO BE MODIFIED
                pi_a += processed_log_pi[i][actions_batch[i]]
            # pi_a = tf.gather(log_pi, actions_batch[:], axis=1, batch_dims=1)
            # pi_a = -tf.reduce_sum(tf.squeeze(deltas_batch) * pi_a)

        grad_pi_a = tape.gradient(pi_a, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grad_pi_a, self.trainable_weights))


class SingleValueFunctionApproximatorBatch(Model):
    def __init__(self, input_shape, optimizer):
        super().__init__()
        self.input_shape = input_shape
        self.optimizer = optimizer
        self.input_a = Input(shape=(self.input_shape))
        self.input_b = Input(shape=(self.input_shape))
        self.dense_1 = layers.Dense(128, activation='tanh')
        self.dense_2 = layers.Dense(256, activation='tanh')
        self.dense_3 = layers.Dense(128, activation='tanh')
        # self.value_function = layers.Dense(1, activation='relu', name="value_function")
        self.value_function = layers.Dense(1, name="value_function")
        self.build_model()

    def preprocess(self, obs_batch):
        if isinstance(obs_batch, Tuple):
            obs_batch = [obs_batch] # to handle the case where obs_batch is a single observation

        obs_1, obs_2 = zip(*obs_batch)
        obs_batch = tf.concat([tf.stack(obs_1), tf.stack(obs_2)], axis=-1)
        return obs_batch


    def call(self, obs_batch, training=False):
        x = self.preprocess(obs_batch)
        x = self.dense_1(x)
        x = self.dense_2(x)
        x = self.dense_3(x)
        value_function = self.value_function(x)
        return value_function

    def build_model(self):
        # computing a forward pass in order to automatically build the model
        dummy_input =  (
            tf.zeros((1, 96)),
            tf.zeros((1, 96))
        )
        _ = self(dummy_input)

    def train_step(self, deltas_batch: tf.Tensor, obs_batch): # deltas is a tf.Tensor of shape (batch_size,1)
        # update w with w + alpha_w*grad_v^(St)*delta
        with tf.GradientTape() as tape:
            state_value = self.call(obs_batch, training=True)
            loss = tf.reduce_mean(tf.square(deltas_batch)) # adding a term that minimizes the distance between the true and the estimated value
            processed_state_value = -deltas_batch * state_value + loss

        grad_state_value = tape.gradient(processed_state_value, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grad_state_value, self.trainable_weights))

class SingleMyAgentBatch(Agent):
    """
    This class implements a single Agent
    For now let's treat it like a single player identified by self.index
    """
    def __init__(self, actor, critic, idx, base_env: OvercookedEnv, epsilon = 0.1):
        super().__init__()
        self.actor = actor
        self.critic = critic
        self.idx = idx
        if not self.idx in [0,1]:
            raise AssertionError("The index of the agent must be either 0 or 1!")
        self.base_env = base_env
        self.epsilon = epsilon

    def action(self, obs):
        """
        obs: preprocessed observation (or overcookedstate)
        We want to output the action given the state. can use a NN!
        should return a tuple (Action, Dict)
        Dict should contain info about the action ('action_probs': numpy array)
        """
        if isinstance(obs, OvercookedState):
            # this is useful for translating the OvercookedState
            # into observation that can be fed into the NN.
            obs_from_state = self.base_env.featurize_state_mdp(obs)
            obs = (obs_from_state[0],obs_from_state[1])


        action_probs = self.actor.call(obs).numpy()
        if np.random.random() > self.epsilon:
            action = Action.argmax(action_probs) # greedy selection
        else:
            action_idx = np.random.choice(range(0,6), size=1)[0]
            action = Action.INDEX_TO_ACTION[action_idx] # random exploration
        return (action, {'action_probs': action_probs})

    def actions(self, obss):
        """
        Look at the documentation of the Agent class
        """
        pass

    def update(self, obs, reward):
        """
        What do we need to update?
        """
        pass

    def decay_epsilon(self, decay):
        if self.epsilon - decay <= 0.5:
            self.epsilon = 0.5
        else:
            self.epsilon -= decay

In [ ]:
number_of_frames = 9
layout_name = "cramped_room"
base_mdp = OvercookedGridworld.from_layout_name(layout_name=layout_name) #or other layout
base_env = OvercookedEnv.from_mdp(base_mdp, info_level=0, horizon=number_of_frames)
env = Overcooked(base_env=base_env, featurize_fn=base_env.featurize_state_mdp)

alpha_t = 1e-5
alpha_w = 1e-5

optimizer_actor_1 = Adam(learning_rate=alpha_t)
optimizer_actor_2 = Adam(learning_rate=alpha_t)
optimizer_critic_1 = Adam(learning_rate=alpha_w)
optimizer_critic_2 = Adam(learning_rate=alpha_w)

input_shape = env.observation_space._shape

single_actor_batch_1 = SinglePolicyBatch(
    input_shape=input_shape,
    num_actions=Action.NUM_ACTIONS,
    optimizer=optimizer_actor_1
)
single_actor_batch_2 = SinglePolicyBatch(
    input_shape=input_shape,
    num_actions=Action.NUM_ACTIONS,
    optimizer=optimizer_actor_2
)

single_critic_batch_1 = SingleValueFunctionApproximatorBatch(
    input_shape=input_shape,
    optimizer=optimizer_critic_1
    )

single_critic_batch_2 = SingleValueFunctionApproximatorBatch(
    input_shape=input_shape,
    optimizer=optimizer_critic_2
    )

agent_1 = SingleMyAgentBatch(
    actor=single_actor_batch_1,
    critic=single_critic_batch_1,
    idx=0,
    base_env=base_env,
    epsilon=0.9
)
agent_2 = SingleMyAgentBatch(
    actor=single_actor_batch_2,
    critic=single_critic_batch_2,
    idx=1,
    base_env=base_env,
    epsilon=0.9
)

In [ ]:
path_critic = "networks/critic/"
path_actor = "networks/actor/"

In [ ]:
number_of_episodes = 50
batch_size = 30
number_of_epochs = 2
average_reward = 0
try:
    for episode in range(1, number_of_episodes + 1):
        timestep = 0
        obs = env.reset()
        done = False

        shaped_reward_factor = 1 # multiplicative factor for discounting shaped reward.
        discount_rate = 1 # will decrease the shaped reward every time
        cumulative_reward = 0

        # if (episode) % 100 == 0:
        #     agent_1.decay_epsilon(decay=0.1)
        #     agent_2.decay_epsilon(decay=0.1)
        #     single_critic_batch_1.save_weights(path_critic + "single_critic_batch_1_"+str(agent_1.epsilon)[2]+".weights.h5")
        #     single_critic_batch_2.save_weights(path_critic + "single_critic_batch_2_"+str(agent_1.epsilon)[2]+".weights.h5")
        #     single_actor_batch_1.save_weights(path_actor + "single_actor_batch_1"+str(agent_2.epsilon)[2]+".weights.h5")
        #     single_actor_batch_2.save_weights(path_actor + "single_actor_batch_2"+str(agent_2.epsilon)[2]+".weights.h5")

        agent_1_actions = [0,3,5,4,4,2,0,5,3]
        agent_2_actions = [2,5,3,0,5,2,4,4,4]
        agent_1_deltas = []
        agent_2_deltas = []
        agent_1_rewards = []
        agent_2_rewards = []
        observations = []

        while not done:
            # reordering the actions based on the POV of the environment
            if obs['other_agent_env_idx'] == 0:
                my_obs = (obs['both_agent_obs'][1],obs['both_agent_obs'][0])
                obs['both_agent_obs'] = my_obs

            # action1 = agent_1.action(obs['both_agent_obs'])
            # action2 = agent_2.action(obs['both_agent_obs'])
            # player_1_action = Action.ACTION_TO_INDEX[action1[0]]
            # player_2_action = Action.ACTION_TO_INDEX[action2[0]]
            if obs['other_agent_env_idx'] == 1:
                # action = (player_1_action,player_2_action)
                action = (agent_1_actions[timestep], agent_2_actions[timestep])
            else:
                # action = (player_2_action,player_1_action)
                action = (agent_2_actions[timestep], agent_1_actions[timestep])

            # agent_1_actions.append(player_1_action)
            # agent_2_actions.append(player_2_action)
            observations.append(obs['both_agent_obs'])

            new_obs, reward, done, env_info = env.step(action)
            # shaped_reward = sum(env_info['shaped_r_by_agent']) # let's use shaped reward for learning how to play first.
            agent_1_shaped_reward = env_info['shaped_r_by_agent'][0]
            agent_2_shaped_reward = env_info['shaped_r_by_agent'][1]

            # if shaped_reward != 0:
            #     shaped_reward = shaped_reward * shaped_reward_factor # discounting the shaped reward
            #     shaped_reward_factor *= discount_rate


            agent_1_total_reward = reward + agent_1_shaped_reward
            agent_2_total_reward = reward + agent_2_shaped_reward
            agent_1_rewards.append(agent_1_total_reward)
            agent_2_rewards.append(agent_2_total_reward)
            cumulative_reward += (agent_1_total_reward + agent_2_total_reward - reward) # since it's in both the total rewards

            obs = new_obs

            timestep += 1

        average_reward = 1/(episode)*( cumulative_reward + (episode-1)*average_reward)
        print(f"Episode [{episode}] terminated at timestep {timestep}. cumulative reward: {cumulative_reward}.")

        print(f"cumulative sparse reward: {env_info['episode']['ep_game_stats']['cumulative_sparse_rewards_by_agent']}")
        print(f"cumulative shaped reward: {env_info['episode']['ep_game_stats']['cumulative_shaped_rewards_by_agent']}")

        print(f"Computing the deltas (MONTE-CARLO)")
        for t in range(len(observations)):
            G_t_agent_1 = np.sum(agent_1_rewards[t:])
            G_t_agent_2 = np.sum(agent_2_rewards[t:])
            agent_1_delta = G_t_agent_1 - single_critic_batch_1.call(observations[t])
            agent_2_delta = G_t_agent_2 - single_critic_batch_2.call(observations[t])
            agent_1_deltas.append(agent_1_delta)
            agent_2_deltas.append(agent_2_delta)

        print(f"Performing stocastic gradient descent with {number_of_epochs} epochs.")
        for epoch in range(number_of_epochs):
            num_batches = len(observations) // batch_size
            shuffled_indices = tf.random.shuffle(tf.range(len(observations)))
            for batch in range(num_batches):
                if batch == num_batches: # last batch
                    idx = shuffled_indices[batch*batch_size:]
                else:
                    idx = shuffled_indices[batch*batch_size:(batch+1)*batch_size]

                agent_1_deltas_batch = tf.squeeze(tf.gather(agent_1_deltas, idx), axis=-1)
                agent_2_deltas_batch = tf.squeeze(tf.gather(agent_2_deltas, idx), axis=-1)
                agent_1_actions_batch = tf.gather(agent_1_actions, idx)
                agent_2_actions_batch = tf.gather(agent_2_actions, idx)
                observations_batch = tf.gather(observations, idx)

                single_critic_batch_1.train_step(agent_1_deltas_batch, observations_batch)
                single_critic_batch_2.train_step(agent_2_deltas_batch, observations_batch)
                single_actor_batch_1.train_step(agent_1_deltas_batch, observations_batch, agent_1_actions_batch)
                single_actor_batch_2.train_step(agent_2_deltas_batch, observations_batch, agent_2_actions_batch)
            print(f"Epoch {epoch+1} terminated.")

except KeyboardInterrupt:
    # single_critic_batch_1.save_weights(path_critic + "single_critic_batch_1_"+str(agent_1.epsilon)[2]+".weights.h5")
    # single_critic_batch_2.save_weights(path_critic + "single_critic_batch_2_"+str(agent_1.epsilon)[2]+".weights.h5")
    # single_actor_batch_1.save_weights(path_actor + "single_actor_batch_1"+str(agent_1.epsilon)[2]+".weights.h5")
    # single_actor_batch_2.save_weights(path_actor + "single_actor_batch_2"+str(agent_1.epsilon)[2]+".weights.h5")
    print("Unexpected interruption of training. Saving weights")

Episode [1] terminated at timestep 9. cumulative reward: 6.
cumulative sparse reward: [0 0]
cumulative shaped reward: [3 3]
Computing the deltas (MONTE-CARLO)
Performing stocastic gradient descent with 2 epochs.
Epoch 1 terminated.
Epoch 2 terminated.
Episode [2] terminated at timestep 9. cumulative reward: 6.
cumulative sparse reward: [0 0]
cumulative shaped reward: [3 3]
Computing the deltas (MONTE-CARLO)
Performing stocastic gradient descent with 2 epochs.
Epoch 1 terminated.
Epoch 2 terminated.
Episode [3] terminated at timestep 9. cumulative reward: 6.
cumulative sparse reward: [0 0]
cumulative shaped reward: [3 3]
Computing the deltas (MONTE-CARLO)
Performing stocastic gradient descent with 2 epochs.
Epoch 1 terminated.
Epoch 2 terminated.
Episode [4] terminated at timestep 9. cumulative reward: 6.
cumulative sparse reward: [0 0]
cumulative shaped reward: [3 3]
Computing the deltas (MONTE-CARLO)
Performing stocastic gradient descent with 2 epochs.
Epoch 1 terminated.
Epoch 2 term

In [ ]:
agent_1.actor(observations)

<tf.Tensor: shape=(9, 6), dtype=float32, numpy=
array([[0.54414874, 0.05493079, 0.1023247 , 0.02552165, 0.04326788,
        0.22980621],
       [0.5266621 , 0.03774102, 0.11036786, 0.02473315, 0.0646782 ,
        0.23581766],
       [0.5498627 , 0.04551601, 0.10392934, 0.03518714, 0.06702498,
        0.19847982],
       [0.41870925, 0.04036696, 0.12767266, 0.06343313, 0.1267527 ,
        0.22306532],
       [0.4566209 , 0.03419267, 0.11589438, 0.04616598, 0.11127401,
        0.23585202],
       [0.39955464, 0.06012763, 0.22791953, 0.03555314, 0.07913763,
        0.19770738],
       [0.60577947, 0.06245914, 0.09557437, 0.00751843, 0.05389806,
        0.17477058],
       [0.5739549 , 0.0505186 , 0.09698042, 0.00787342, 0.05436873,
        0.21630393],
       [0.6620109 , 0.04429185, 0.09476521, 0.00352296, 0.03521813,
        0.16019094]], dtype=float32)>

In [ ]:
single_critic_batch_1.load_weights(path_critic + "single_critic_batch_1_6.weights.h5")
single_critic_batch_2.load_weights(path_critic + "single_critic_batch_2_6.weights.h5")
single_actor_batch_1.load_weights(path_actor + "single_actor_batch_16.weights.h5")
single_actor_batch_2.load_weights(path_actor + "single_actor_batch_26.weights.h5")

In [ ]:
# Here we create an evaluator for the cramped_room layout
layout = "cramped_room"
ae = AgentEvaluator.from_layout_name(mdp_params={"layout_name": layout, "old_dynamics": True},
                                     env_params={"horizon": 400})

agent_1.epsilon = 0.1
agent_2.epsilon = 0.1
ap = AgentPair(agent_1, agent_2)

trajs = ae.evaluate_agent_pair(ap, 10)

StateVisualizer().display_rendered_trajectory(trajs, ipython_display=True)

Avg rew: 0.00 (std: 0.00, se: 0.00); avg len: 400.00; : 100%|██████████| 10/10 [00:44<00:00,  4.47s/it]


interactive(children=(IntSlider(value=0, description='timestep', max=399), Output()), _dom_classes=('widget-in…

In [ ]:
# Here we create an evaluator for the cramped_room layout
layout = "cramped_room"
ae = AgentEvaluator.from_layout_name(mdp_params={"layout_name": layout, "old_dynamics": True},
                                     env_params={"horizon": 400})

agent_1.epsilon = 0.1
agent_2.epsilon = 0.1
ap = AgentPair(agent_2, agent_1)

trajs = ae.evaluate_agent_pair(ap, 10)

StateVisualizer().display_rendered_trajectory(trajs, ipython_display=True)

Avg rew: 0.00 (std: 0.00, se: 0.00); avg len: 400.00; : 100%|██████████| 10/10 [00:46<00:00,  4.63s/it]


interactive(children=(IntSlider(value=0, description='timestep', max=399), Output()), _dom_classes=('widget-in…

# PPO Implementation with different NNs for the agents:

In [ ]:
class SinglePPOPolicyBatch(Model):
    def __init__(self, input_shape, num_actions, optimizer, epsilon = 0.05):
        super().__init__()
        self.input_shape = input_shape
        self.num_actions = num_actions
        self.optimizer = optimizer
        self.epsilon = epsilon
        self.input_a = Input(shape=(self.input_shape))
        self.input_b = Input(shape=(self.input_shape))
        self.dense_1 = layers.Dense(128, activation='tanh')
        self.dense_2 = layers.Dense(256, activation='tanh')
        self.dense_3 = layers.Dense(128, activation='tanh')
        self.policy = layers.Dense(self.num_actions, activation='softmax', name="policy")
        self.build_model()

    def preprocess(self, obs_batch):
        if isinstance(obs_batch, Tuple):
            obs_batch = [obs_batch] # to handle the case where obs_batch is a single observation

        obs_1, obs_2 = zip(*obs_batch)
        obs_batch = tf.concat([tf.stack(obs_1), tf.stack(obs_2)], axis=-1)
        return obs_batch


    def call(self, obs_batch, training=False):
        x = self.preprocess(obs_batch)
        x = self.dense_1(x)
        x = self.dense_2(x)
        x = self.dense_3(x)
        policy = self.policy(x)
        return policy

    def build_model(self):
        # computing a forward pass in order to automatically build the model
        dummy_input = (
            tf.zeros((1, 96)),
            tf.zeros((1, 96))
        )
        _ = self(dummy_input)

    def train_step(self, deltas_batch: tf.Tensor, obs_batch, actions_batch, old_policy):

        with tf.GradientTape() as tape:
            pi = self.call(obs_batch, training=True)
            old_pi = old_policy.call(obs_batch)
            pi_ratio = pi / old_pi + 1e-8 # to avoid numerical instability
            pi_clipped_ratio = tf.clip_by_value(pi_ratio, 1 - self.epsilon, 1 + self.epsilon)
            pi_ratio_advantage = pi_ratio*deltas_batch
            pi_clipped_ratio_advantage = pi_clipped_ratio*deltas_batch
            L = 0
            for i in range(len(actions_batch)):
                L += min(pi_ratio_advantage[i][actions_batch[i]], pi_clipped_ratio_advantage[i][actions_batch[i]])
            loss = -L

        grad_loss = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grad_loss, self.trainable_weights))


class SinglePPOValueFunctionApproximatorBatch(Model):
    def __init__(self, input_shape, optimizer):
        super().__init__()
        self.input_shape = input_shape
        self.optimizer = optimizer
        self.input_a = Input(shape=(self.input_shape))
        self.input_b = Input(shape=(self.input_shape))
        self.dense_1 = layers.Dense(128, activation='tanh')
        self.dense_2 = layers.Dense(256, activation='tanh')
        self.dense_3 = layers.Dense(128, activation='tanh')
        # self.value_function = layers.Dense(1, activation='relu', name="value_function")
        self.value_function = layers.Dense(1, name="value_function")
        self.build_model()

    def preprocess(self, obs_batch):
        if isinstance(obs_batch, Tuple):
            obs_batch = [obs_batch] # to handle the case where obs_batch is a single observation

        obs_1, obs_2 = zip(*obs_batch)
        obs_batch = tf.concat([tf.stack(obs_1), tf.stack(obs_2)], axis=-1)
        return obs_batch


    def call(self, obs_batch, training=False):
        x = self.preprocess(obs_batch)
        x = self.dense_1(x)
        x = self.dense_2(x)
        x = self.dense_3(x)
        value_function = self.value_function(x)
        return value_function

    def build_model(self):
        # computing a forward pass in order to automatically build the model
        dummy_input =  (
            tf.zeros((1, 96)),
            tf.zeros((1, 96))
        )
        _ = self(dummy_input)

    def train_step(self, deltas_batch: tf.Tensor, obs_batch): # deltas is a tf.Tensor of shape (batch_size,1)
        # update w with w + alpha_w*grad_v^(St)*delta
        with tf.GradientTape() as tape:
            state_value = self.call(obs_batch, training=True)
            loss = tf.reduce_mean(tf.square(deltas_batch)) # adding a term that minimizes the distance between the true and the estimated value
            processed_state_value = -deltas_batch * state_value + loss

        grad_state_value = tape.gradient(processed_state_value, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grad_state_value, self.trainable_weights))

class SinglePPOMyAgentBatch(Agent):
    """
    This class implements a single Agent
    For now let's treat it like a single player identified by self.index
    """
    def __init__(self, actor, old_policy, critic, idx, base_env: OvercookedEnv, epsilon = 0.1):
        super().__init__()
        self.actor = actor
        self.old_policy = old_policy
        self.critic = critic
        self.idx = idx
        if not self.idx in [0,1]:
            raise AssertionError("The index of the agent must be either 0 or 1!")
        self.base_env = base_env
        self.epsilon = epsilon

    def action(self, obs):
        """
        obs: preprocessed observation (or overcookedstate)
        We want to output the action given the state. can use a NN!
        should return a tuple (Action, Dict)
        Dict should contain info about the action ('action_probs': numpy array)
        """
        if isinstance(obs, OvercookedState):
            # this is useful for translating the OvercookedState
            # into observation that can be fed into the NN.
            obs_from_state = self.base_env.featurize_state_mdp(obs)
            obs = (obs_from_state[0],obs_from_state[1])


        action_probs = self.actor.call(obs).numpy()
        if np.random.random() > self.epsilon:
            action = Action.argmax(action_probs) # greedy selection
        else:
            action_idx = np.random.choice(range(0,6), size=1)[0]
            action = Action.INDEX_TO_ACTION[action_idx] # random exploration
        return (action, {'action_probs': action_probs})

    def actions(self, obss):
        """
        Look at the documentation of the Agent class
        """
        pass

    def update_old_policy(self):
        self.old_policy.set_weights(self.actor.get_weights())


    def decay_epsilon(self, decay):
        if self.epsilon - decay <= 0.5:
            self.epsilon = 0.5
        else:
            self.epsilon -= decay

In [ ]:
number_of_frames = 9
layout_name = "cramped_room"
base_mdp = OvercookedGridworld.from_layout_name(layout_name=layout_name) #or other layout
base_env = OvercookedEnv.from_mdp(base_mdp, info_level=0, horizon=number_of_frames)
env = Overcooked(base_env=base_env, featurize_fn=base_env.featurize_state_mdp)

alpha_t = 1e-6
alpha_w = 1e-3

optimizer_actor_1 = Adam(learning_rate=alpha_t)
optimizer_actor_2 = Adam(learning_rate=alpha_t)
optimizer_critic_1 = Adam(learning_rate=alpha_w)
optimizer_critic_2 = Adam(learning_rate=alpha_w)

input_shape = env.observation_space._shape

single_actor_batch_1 = SinglePPOPolicyBatch(
    input_shape=input_shape,
    num_actions=Action.NUM_ACTIONS,
    optimizer=optimizer_actor_1
)
single_actor_batch_2 = SinglePPOPolicyBatch(
    input_shape=input_shape,
    num_actions=Action.NUM_ACTIONS,
    optimizer=optimizer_actor_2
)
old_policy_1 = SinglePPOPolicyBatch(
    input_shape=input_shape,
    num_actions=Action.NUM_ACTIONS,
    optimizer=None
)
old_policy_2 = SinglePPOPolicyBatch(
    input_shape=input_shape,
    num_actions=Action.NUM_ACTIONS,
    optimizer=None
)

single_critic_batch_1 = SinglePPOValueFunctionApproximatorBatch(
    input_shape=input_shape,
    optimizer=optimizer_critic_1
    )

single_critic_batch_2 = SinglePPOValueFunctionApproximatorBatch(
    input_shape=input_shape,
    optimizer=optimizer_critic_2
    )

agent_1 = SinglePPOMyAgentBatch(
    actor=single_actor_batch_1,
    old_policy=old_policy_1,
    critic=single_critic_batch_1,
    idx=0,
    base_env=base_env,
    epsilon=0.9
)
agent_2 = SinglePPOMyAgentBatch(
    actor=single_actor_batch_2,
    old_policy=old_policy_2,
    critic=single_critic_batch_2,
    idx=1,
    base_env=base_env,
    epsilon=0.9
)

agent_1.update_old_policy()
agent_2.update_old_policy()

In [ ]:
path_critic = "networks/critic/"
path_actor = "networks/actor/"

- try to learn only the valuefunction first.
- try to build more complex neural networks
- you need to sample to have the action!

In [ ]:
number_of_episodes = 100
batch_size = 9
number_of_epochs = 2
average_reward = 0
try:
    for episode in range(1, number_of_episodes + 1):
        timestep = 0
        obs = env.reset()
        done = False

        shaped_reward_factor = 1 # multiplicative factor for discounting shaped reward.
        discount_rate = 1 # will decrease the shaped reward every time
        cumulative_reward = 0

        # if (episode) % 100 == 0:
        #     agent_1.decay_epsilon(decay=0.1)
        #     agent_2.decay_epsilon(decay=0.1)
        #     single_critic_batch_1.save_weights(path_critic + "PPO_single_critic_batch_1_"+str(agent_1.epsilon)[2]+".weights.h5")
        #     single_critic_batch_2.save_weights(path_critic + "PPO_single_critic_batch_2_"+str(agent_1.epsilon)[2]+".weights.h5")
        #     single_actor_batch_1.save_weights(path_actor + "PPO_single_actor_batch_1"+str(agent_2.epsilon)[2]+".weights.h5")
        #     single_actor_batch_2.save_weights(path_actor + "PPO_single_actor_batch_2"+str(agent_2.epsilon)[2]+".weights.h5")

        agent_1_actions = [0,3,5,4,4,2,0,5,3]
        agent_2_actions = [2,5,3,0,5,2,4,4,4]
        agent_1_deltas = []
        agent_2_deltas = []
        # agent_1_deltas = [0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]
        # agent_2_deltas = [0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]
        agent_1_rewards = []
        agent_2_rewards = []
        observations = []

        while not done:
            # render = env.render() / 255.0
            # plt.imshow(render)
            # plt.show()

            # reordering the actions based on the POV of the environment
            if obs['other_agent_env_idx'] == 0:
                my_obs = (obs['both_agent_obs'][1],obs['both_agent_obs'][0])
                obs['both_agent_obs'] = my_obs

            # action1 = agent_1.action(obs['both_agent_obs'])
            # action2 = agent_2.action(obs['both_agent_obs'])
            # player_1_action = Action.ACTION_TO_INDEX[action1[0]]
            # player_2_action = Action.ACTION_TO_INDEX[action2[0]]

            if obs['other_agent_env_idx'] == 1:
                # action = (player_1_action,player_2_action)
                action = (agent_1_actions[timestep], agent_2_actions[timestep])
            else:
                # action = (player_2_action,player_1_action)
                action = (agent_2_actions[timestep], agent_1_actions[timestep])

            # agent_1_actions.append(player_1_action)
            # agent_2_actions.append(player_2_action)
            observations.append(obs['both_agent_obs'])

            new_obs, reward, done, env_info = env.step(action)
            # shaped_reward = sum(env_info['shaped_r_by_agent']) # let's use shaped reward for learning how to play first.
            agent_1_shaped_reward = env_info['shaped_r_by_agent'][0]
            agent_2_shaped_reward = env_info['shaped_r_by_agent'][1]

            # if shaped_reward != 0:
            #     shaped_reward = shaped_reward * shaped_reward_factor # discounting the shaped reward
            #     shaped_reward_factor *= discount_rate


            agent_1_total_reward = reward + agent_1_shaped_reward
            agent_2_total_reward = reward + agent_2_shaped_reward
            agent_1_rewards.append(agent_1_total_reward)
            agent_2_rewards.append(agent_2_total_reward)
            cumulative_reward += (agent_1_total_reward + agent_2_total_reward - reward) # since it's in both the total rewards

            obs = new_obs

            timestep += 1

        average_reward = 1/(episode)*( cumulative_reward + (episode-1)*average_reward)
        print(f"Episode [{episode}] terminated at timestep {timestep}. cumulative reward: {cumulative_reward}.")

        print(f"cumulative sparse reward: {env_info['episode']['ep_game_stats']['cumulative_sparse_rewards_by_agent']}")
        print(f"cumulative shaped reward: {env_info['episode']['ep_game_stats']['cumulative_shaped_rewards_by_agent']}")

        print(f"Computing the deltas (MONTE-CARLO)")
        for t in range(len(observations)):
            G_t_agent_1 = np.sum(agent_1_rewards[t:])
            G_t_agent_2 = np.sum(agent_2_rewards[t:])
            agent_1_delta = G_t_agent_1 - single_critic_batch_1.call(observations[t])
            agent_2_delta = G_t_agent_2 - single_critic_batch_2.call(observations[t])
            agent_1_deltas.append(agent_1_delta)
            agent_2_deltas.append(agent_2_delta)

        print(f"Performing stocastic gradient descent with {number_of_epochs} epochs.")
        for epoch in range(number_of_epochs):
            num_batches = len(observations) // batch_size
            shuffled_indices = tf.random.shuffle(tf.range(len(observations)))
            for batch in range(num_batches):
                if batch == num_batches: # last batch
                    idx = shuffled_indices[batch*batch_size:]
                else:
                    idx = shuffled_indices[batch*batch_size:(batch+1)*batch_size]

                agent_1_deltas_batch = tf.squeeze(tf.gather(agent_1_deltas, idx), axis=-1)
                agent_2_deltas_batch = tf.squeeze(tf.gather(agent_2_deltas, idx), axis=-1)
                agent_1_actions_batch = tf.gather(agent_1_actions, idx)
                agent_2_actions_batch = tf.gather(agent_2_actions, idx)
                observations_batch = tf.gather(observations, idx)

                single_critic_batch_1.train_step(agent_1_deltas_batch, observations_batch)
                single_critic_batch_2.train_step(agent_2_deltas_batch, observations_batch)
                single_actor_batch_1.train_step(agent_1_deltas_batch, observations_batch, agent_1_actions_batch, agent_1.old_policy)
                single_actor_batch_2.train_step(agent_2_deltas_batch, observations_batch, agent_2_actions_batch, agent_2.old_policy)

            print(f"Epoch {epoch+1} terminated.")
        agent_1.update_old_policy()
        agent_2.update_old_policy()
        print("Old policy updated.")

except KeyboardInterrupt:
    # single_critic_batch_1.save_weights(path_critic + "PPO_single_critic_batch_1_"+str(agent_1.epsilon)[2]+".weights.h5")
    # single_critic_batch_2.save_weights(path_critic + "PPO_single_critic_batch_2_"+str(agent_1.epsilon)[2]+".weights.h5")
    # single_actor_batch_1.save_weights(path_actor + "PPO_single_actor_batch_1"+str(agent_1.epsilon)[2]+".weights.h5")
    # single_actor_batch_2.save_weights(path_actor + "PPO_single_actor_batch_2"+str(agent_1.epsilon)[2]+".weights.h5")
    print("Unexpected interruption of training. Saving weights")

Episode [1] terminated at timestep 9. cumulative reward: 6.
cumulative sparse reward: [0 0]
cumulative shaped reward: [3 3]
Computing the deltas (MONTE-CARLO)
Performing stocastic gradient descent with 2 epochs.
Epoch 1 terminated.
Epoch 2 terminated.
Old policy updated.
Episode [2] terminated at timestep 9. cumulative reward: 6.
cumulative sparse reward: [0 0]
cumulative shaped reward: [3 3]
Computing the deltas (MONTE-CARLO)
Performing stocastic gradient descent with 2 epochs.
Epoch 1 terminated.
Epoch 2 terminated.
Old policy updated.
Episode [3] terminated at timestep 9. cumulative reward: 6.
cumulative sparse reward: [0 0]
cumulative shaped reward: [3 3]
Computing the deltas (MONTE-CARLO)
Performing stocastic gradient descent with 2 epochs.
Epoch 1 terminated.
Epoch 2 terminated.
Old policy updated.
Episode [4] terminated at timestep 9. cumulative reward: 6.
cumulative sparse reward: [0 0]
cumulative shaped reward: [3 3]
Computing the deltas (MONTE-CARLO)
Performing stocastic grad

In [ ]:
print(f"deltas: {agent_1_deltas}")
print(f"actions: {agent_1_actions}")

deltas: [<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.08810043]], dtype=float32)>, <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.57649255]], dtype=float32)>, <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.74333763]], dtype=float32)>, <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.4717548]], dtype=float32)>, <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.48167014]], dtype=float32)>, <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.4180901]], dtype=float32)>, <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.20407844]], dtype=float32)>, <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.14600158]], dtype=float32)>, <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-2.7087955]], dtype=float32)>]
actions: [0, 3, 5, 4, 4, 2, 0, 5, 3]


In [ ]:
agent_1.critic(observations)

<tf.Tensor: shape=(9, 1), dtype=float32, numpy=
array([[3.0029888],
       [3.476031 ],
       [3.6558423],
       [3.420232 ],
       [3.426095 ],
       [3.357685 ],
       [3.0948834],
       [3.0314775],
       [2.5496166]], dtype=float32)>

In [ ]:
print(agent_1_actions)
agent_1.actor(observations)

[0, 3, 5, 4, 4, 2, 0, 5, 3]


<tf.Tensor: shape=(9, 6), dtype=float32, numpy=
array([[0.42995033, 0.15540141, 0.07245072, 0.09965793, 0.08214606,
        0.16039355],
       [0.3812262 , 0.11570279, 0.05563588, 0.09430336, 0.1404916 ,
        0.2126403 ],
       [0.39994976, 0.14705846, 0.05766393, 0.07857709, 0.10975736,
        0.2069934 ],
       [0.38749117, 0.17783736, 0.10564444, 0.07958111, 0.07753027,
        0.17191567],
       [0.4107398 , 0.1536756 , 0.10426272, 0.08185078, 0.08316261,
        0.16630848],
       [0.37289113, 0.15720315, 0.08807483, 0.06117241, 0.08681408,
        0.2338444 ],
       [0.2669122 , 0.13264365, 0.07348461, 0.10169731, 0.24469541,
        0.18056683],
       [0.25690734, 0.12257473, 0.06750435, 0.08076221, 0.28243768,
        0.18981364],
       [0.28856304, 0.09113441, 0.06223999, 0.08719691, 0.28227845,
        0.18858723]], dtype=float32)>

In [ ]:
number_of_epochs = 2
print(f"Performing stocastic gradient descent with {number_of_epochs} epochs.")
for epoch in range(number_of_epochs):
    num_batches = len(observations) // batch_size
    shuffled_indices = tf.random.shuffle(tf.range(len(observations)))
    for batch in range(num_batches):
        if batch == num_batches: # last batch
            idx = shuffled_indices[batch*batch_size:]
        else:
            idx = shuffled_indices[batch*batch_size:(batch+1)*batch_size]

        agent_1_deltas_batch = tf.squeeze(tf.gather(agent_1_deltas, idx), axis=-1)
        agent_2_deltas_batch = tf.squeeze(tf.gather(agent_2_deltas, idx), axis=-1)
        agent_1_actions_batch = tf.gather(agent_1_actions, idx)
        agent_2_actions_batch = tf.gather(agent_2_actions, idx)
        observations_batch = tf.gather(observations, idx)

        single_critic_batch_1.train_step(agent_1_deltas_batch, observations_batch)
        single_critic_batch_2.train_step(agent_2_deltas_batch, observations_batch)
        single_actor_batch_1.train_step(agent_1_deltas_batch, observations_batch, agent_1_actions_batch, agent_1.old_policy)
        single_actor_batch_2.train_step(agent_2_deltas_batch, observations_batch, agent_2_actions_batch, agent_2.old_policy)

    print(f"Epoch {epoch+1} terminated.")
agent_1.update_old_policy()
agent_2.update_old_policy()
print("Old policy updated.")

Performing stocastic gradient descent with 2 epochs.
Epoch 1 terminated.
Epoch 2 terminated.
Old policy updated.


In [ ]:
print(f"actions: {agent_1_actions}")
print(agent_1.actor(observations))
# [0.09367532, 0.17965837, 0.3680178 , 0.12918349, 0.12197147, 0.10749362] 0
# [0.10268684, 0.14741758, 0.35757372, 0.16343038, 0.13853838,0.09035313] 3
# [0.11685525, 0.14133197, 0.28310293, 0.20308046, 0.17950246,0.07612696],5,
# [0.11804521, 0.17394674, 0.19776204, 0.17337577, 0.26236528,0.07450493],4,
# [0.12199399, 0.16056925, 0.19554548, 0.16675068, 0.27091032,0.08423036],4,
# [0.14682746, 0.20350426, 0.17738527, 0.1450429 , 0.26738393,0.05985614], 2,
# [0.13215278, 0.18081923, 0.24620707, 0.16264597, 0.17406137,0.10411353], 0,
# [0.13127315, 0.18660071, 0.24234423, 0.14297846, 0.17740306,0.11940035],5,
# [0.13906823, 0.16432595, 0.35385984, 0.13538487, 0.11182725,0.09553384] 3


actions: [0, 3, 5, 4, 4, 2, 0, 5, 3]
tf.Tensor(
[[0.15358098 0.12851276 0.19724137 0.26844463 0.09837393 0.15384635]
 [0.1419473  0.12488297 0.20040582 0.24967888 0.08087178 0.20221326]
 [0.1426429  0.14456539 0.22539036 0.1953097  0.10137178 0.1907199 ]
 [0.09734543 0.14189063 0.35803217 0.14960736 0.11195736 0.14116707]
 [0.1094325  0.14722984 0.33201593 0.16180198 0.09866285 0.150857  ]
 [0.10592322 0.14850427 0.34247446 0.19748576 0.08631345 0.11929888]
 [0.10938568 0.17282152 0.19826694 0.28383833 0.07338683 0.16230066]
 [0.11258917 0.18495543 0.16243964 0.33746386 0.04965557 0.15289621]
 [0.13929448 0.15747339 0.15320216 0.31434095 0.05044907 0.18524003]], shape=(9, 6), dtype=float32)


In [ ]:
# # Here we create an evaluator for the cramped_room layout
# layout = "cramped_room"
# ae = AgentEvaluator.from_layout_name(mdp_params={"layout_name": layout, "old_dynamics": True},
#                                      env_params={"horizon": 400})

# agent_1.epsilon = 0.5
# agent_2.epsilon = 0.5
# ap = AgentPair(agent_1, agent_2)

# trajs = ae.evaluate_agent_pair(ap, 10)

# StateVisualizer().display_rendered_trajectory(trajs, ipython_display=True)

In [ ]:
# # Here we create an evaluator for the cramped_room layout
# layout = "cramped_room"
# ae = AgentEvaluator.from_layout_name(mdp_params={"layout_name": layout, "old_dynamics": True},
#                                      env_params={"horizon": 400})

# ap = AgentPair(RandomAgent(all_actions=True), RandomAgent(all_actions=True))

# trajs = ae.evaluate_agent_pair(ap, 10)

# StateVisualizer().display_rendered_trajectory(trajs, ipython_display=True)

In [ ]:
agent_1_deltas_batch = tf.squeeze(agent_1_deltas[:5], axis=-1)
agent_2_deltas_batch = tf.squeeze(agent_2_deltas[:5], axis=-1)
agent_1_actions_batch = agent_1_actions[:5]
agent_2_actions_batch = agent_2_actions[:5]
observations_batch = observations[:5]
print(f"delta: {agent_1_deltas_batch}")
print(f"action: {agent_1_actions_batch}")

delta: [[-0.08628636]
 [-0.08628636]
 [-0.08628636]
 [ 0.25002515]
 [-0.05459686]]
action: [5, 4, 5, 2, 4]


In [ ]:
agent_1.actor(observations[:5])

<tf.Tensor: shape=(5, 6), dtype=float32, numpy=
array([[0.1851859 , 0.31293392, 0.12289874, 0.09092614, 0.19850704,
        0.08954828],
       [0.24177429, 0.29987434, 0.08277841, 0.11627084, 0.19729319,
        0.06200891],
       [0.2519065 , 0.32475212, 0.10912957, 0.11884841, 0.1412006 ,
        0.05416276],
       [0.16171499, 0.3840429 , 0.12999965, 0.12385841, 0.14538556,
        0.05499851],
       [0.20342645, 0.36186278, 0.13344555, 0.12230352, 0.13379931,
        0.04516245]], dtype=float32)>

In [ ]:
single_critic_batch_1.train_step(agent_1_deltas_batch, observations_batch)
single_critic_batch_2.train_step(agent_2_deltas_batch, observations_batch)
single_actor_batch_1.train_step(agent_1_deltas_batch, observations_batch, agent_1_actions_batch, agent_1.old_policy)
single_actor_batch_2.train_step(agent_2_deltas_batch, observations_batch, agent_2_actions_batch, agent_2.old_policy)
# agent_1.update_old_policy()

In [ ]:
agent_1.actor(observations[:5])

<tf.Tensor: shape=(5, 6), dtype=float32, numpy=
array([[0.17644215, 0.22335747, 0.24774687, 0.05928389, 0.19960198,
        0.09356767],
       [0.17644215, 0.22335747, 0.24774687, 0.05928389, 0.19960198,
        0.09356767],
       [0.17644215, 0.22335747, 0.24774687, 0.05928389, 0.19960198,
        0.09356767],
       [0.19349185, 0.26201612, 0.21033107, 0.06349459, 0.17717108,
        0.09349547],
       [0.12860686, 0.2923308 , 0.27677163, 0.05776837, 0.13853319,
        0.10598914]], dtype=float32)>